In [8]:
from cfg import *

## readtable function
### Author: Nestor Timonidis   
### Description:   
Given the file pointer of a .csv file, this function stores the csv elements into a dictionary
*        #### input:    
fp = the file pointer corresponding to the .csv file
*        #### output:   
T = the output dictionary

In [12]:
def readtable(fp):
  r = csv.reader(fp)
  keys = r.next()
  T = [ [] for k in keys ]
  for row in r:
    for i,v in enumerate(row):T[i].append(v)
  T = {keys[i]:col for i,col in enumerate(T)}
  return T

In [107]:
def CellTypeParser(filename):
    fp = open(filename,'rb')
    exclude_list = ['Probe', 'Gene.Symbol', 'GeneNames', 'GOTerms', 'GemmaIDs', 'NCBIids']
    forbidden_list = []
    X = []
    gene_annot = []
    cell_annot = []
    # Part I: get genetic and cellular annotations
    reader = csv.reader(fp)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    for idx,row in enumerate(reader):
      if idx == 0:
          for idx2,col in enumerate(row):  
               if idx2 > 0 and col not in exclude_list:
                  cell_annot.append(col)
               elif col in exclude_list:
                   forbidden_list.append(idx2)     
      elif idx > 0:    
           gene_annot.append(row[1])
    
    # Part II: acquire the cell-type specific data
    fp = open(filename,'rb')
    reader = csv.reader(fp)
    for row_idx,row in enumerate(reader):
        #expressions = [val for val in row if val.isnumeric()]
        if row_idx > 0:
           expressionz = [val.replace('NA','NaN',1) for val in row]		
           expressions = [val for idx,val in enumerate(expressionz) if (val.replace('.','',1).isdigit() == True or val == 'NaN') and idx not in forbidden_list]
           X.append(expressions)
            
    X = np.asfortranarray(X,dtype = 'float32')
    fp.close()      
    return X,gene_annot,cell_annot 

In [13]:
class sbaInterface_class:
    def __init__(this,sbaUrl,sbaHost):
        # In Javascript, load sbaInterface.js and create a new sbaInterface object
        display(Javascript("""
            var script = document.createElement('script');
            script.src = '{}/../js/sba-interface.js';
            script.onload = function() {{
              window.global_sbaInterface = new sbaInterface_class('{}');
            }}
            document.head.appendChild(script)
        """.format(sbaHost,sbaUrl)))
        clear_output()
        
    def send(this,sbaCommand):
        display(Javascript("""window.global_sbaInterface.send({})""".format(json_encode(sbaCommand))))
        clear_output() # prevents clogging the Jupyter notebook

## RemoveOutliers class
### Author: Nestor Timonidis   
### Description:   
Removes outliers based on the input array's per column interquartile range. Elements exceeding the range of $(Q1 - 1.5*iqr, Q3 + 1.5*iqr)$,  
where the interquartile range (iqr) and the two quartiles (1st and 3rd) are defined per column of the array, are detected as outliers, and their corresponding columns are being removed from the array. A corresponds to the updated array.
*   ### functions:   
       *        #### init:    
initalizes the quartiles used for estimating outliers. By default are Q1 (1st) and Q3 (3rd)
       *        #### fit:   
scans the input array per column and checks for outliers based on the method provided in the description.
For each detected outlier, the column corresponding to it is removed from the array.
The function returns the updated array, as well as a list that stores the outlier indices.


In [14]:
class RemoveOutliers():
    __slots__ = ['q1','q2'] 
    def __init__(self, q1 = 25, q2 = 75):
        self.q1 = q1
        self.q2 = q2
    def fit(self, A):
        ranges = iqr(A, axis = 0)
        Q1 = np.percentile(A, self.q1, axis = 0)
        Q3 = np.percentile(A, self.q2, axis = 0)

        genes4deletion = Set()
        for row,cell in enumerate(A):
            for col,gene in enumerate(cell):
               #if gene >= Q3[col] + ranges[col]*3/2 or gene <= Q1[col] - ranges[col]*3/2:
                if gene >= Q3[col] + ranges[col]*3/2 or gene <= - Q1[col] + ranges[col]*3/2:
                   genes4deletion.add(col)
                   break

        outliers = list(genes4deletion)
        A = np.delete(A,outliers,1)
        return A,outliers

## filter_cortex function
### Author: Nestor Timonidis   
### Description:   
Given a mask corresponding to specific stuctures of the mouse cortex and a tract tracing experiment defined in a volumetric space, this function filters out voxels of the experiment not corresponding to the mask.
*        #### input:    
         * struct_idx_dict = dictionary that maps identities of anatomically distinct structures, as provided by the Allen Institute, to their position in a unionized space, meaning a 2D space of anatomical structures and brain related experiments.
         * l_vals = the anatomical structures that will be used to filter the data, defined by their position in the unionized space.
         * Annotation = the Annotation identities of the structures in the volumetric space, as provided by the Allen Institute.
         * pd: the tract tracing experiment defined in the volumetric space, measured by projection density.
*        #### output:   
         * l_voxel_mask = the refined volumetric data based on the applied mask.

In [15]:
def filter_cortex(struct_idx_dict,l_vals, Annotation, pd):
        scis = [sci for sci,x in struct_idx_dict.items() if x in l_vals]
        l_mask = []
        for sci in scis:
            tmp = Annotation == int(sci) 
            if l_mask == []:
                l_mask = tmp
            else:
                l_mask = np.ma.mask_or(l_mask,tmp)   

        l_voxel_mask = np.zeros(np.shape(pd))
        l_nzero      = np.nonzero(l_mask)
        l_voxel_mask[l_nzero] = pd[l_nzero]
        return l_voxel_mask
    

## MakeArrayBorders function
### Author: Nestor Timonidis   
### Description:   
This function creates a structural border mask given the annotation volume of the mouse brain, based on an anatomical parcelation made by the Allen Institute. The voxels are masked based on the ones that differ in value with their neighboring voxels and the rest.
*        #### input:    
         * Annotation_copy = annotation volume of the mouse brain (defined in micrometers).
*        #### output:   
         * Annotation_copy2 = the structural border mask

In [16]:
def MakeArrayBorders(Annotation_copy):
    #description: given a matrix, comparison with neighbors so that different neighboring pixels get the value 1 
    #             and create borders, else 0
    
    Annotation_copy2 = np.zeros(np.shape(Annotation_copy))
    
    for x,row in enumerate(Annotation_copy):
       
        for y,col in enumerate(row):
            flag = 0
            if x > 0:
                if (Annotation_copy[x,y] - Annotation_copy[x-1,y]) != 0:
                    flag = 1
            if x < np.shape(Annotation_copy)[0] - 1:
                if (Annotation_copy[x,y] - Annotation_copy[x+1,y]) != 0:
                    flag = 1
            if y > 0:
                if (Annotation_copy[x,y] - Annotation_copy[x,y-1]) != 0:
                    flag = 1
            if y < np.shape(Annotation_copy)[1] - 1:
                if (Annotation_copy[x,y] - Annotation_copy[x,y+1]) != 0:
                    flag = 1    
                    
            if flag == 1:
                Annotation_copy2[x,y] = 1
            else:
                Annotation_copy2[x,y] = 0
                
    return Annotation_copy2          

## BrainPlotter class
### Author: Nestor Timonidis   
### Description:   
This class contains functions that provide visualizations of the mouse brain, either in the form of brain slices or cortical flatmaps. The inputs of interest for visualization are identities of tract tracing or gene expression experiments, as defined by the Allen Institute.
### init function:   
   #### Description:  
creates a brain plotter object by defining a number of parameters for the visualization process.
  *  #### input:  
      *  exp_id = the identity of the experiment  
      *  driver  =  the type of the experiment. It could be wild_type (tract tracing), or gene_expression for the equivalent experiment categories, or the name of a cre category (tract tracing).
      * resolution = resolution of the experiment in micrometers. Acceptable options are: 100, 25 and 10 for tract tracing experiments, and 200 for gene expression.
      

In [17]:
class BrainPlotter:
    
    __slots__ = ['id','driver','pd','resolution']
    def __init__(self,  exp_id = 181599674, driver = 'wild_type', resolution = 25):
        
        self.id         = exp_id
        self.resolution = resolution
        self.driver     = driver  

## BrainPlotter class
### ReduceToParent function
### Author: Nestor Timonidis
### Description:   
In order to make a cortical flatmap visualization for the mouse cortex,    
we need to initially ensure that we have reduced the complexity  
of structural brain areas to a layer inspecific level that corresponds    
to clear anatomically distinct areas like the primary visual or motor cortex.   
Therefore we utilize the MouseConnectivityCache tool to get the ontology    
tree of brain areas, then we reduce it by the isocortex and for each    
isocortical area, we take the identity of its father node, or an area  
belonging up in the tree hierarchy.      
Having achieved that, the next step is to replace the identity of each    
area in the annotation volume structure by the father identity.  
* #### Input: 
  * Annotation = volumetric array containing information about the annotation volume of each voxel, as defined by the Allen Institute for Brain Science.
  * mcc = an instance of mouse connectivity cache class of the same toolbox provided by the Allen Institute, whose functions will be used throughout the ReduceToParent function.
* #### Output:
  * Annotation_copy = the updated annotation volume, with structures having been reduced to their parent node.

In [18]:
class BrainPlotter(BrainPlotter): 
    
    def ReduceToParent(self, Annotation, mcc):
        # grab the StructureTree instance
        structure_tree = mcc.get_structure_tree()
        oapi           = OntologiesApi()
        # Get isocortex summary structures
        kid_to_father = {}
        summary_structures = structure_tree.get_structures_by_set_id([688152357])
        for val in summary_structures:
            kids = structure_tree.descendant_ids([val['id']])
            for kid in kids[0]:
                kid_to_father[kid] = val['id']

        # For each distinct isocortical area, replace its children annotation with the area's id
        father_acros_indices = {val['acronym']:val['id'] for val in summary_structures}

        #store the original annotation shape because it will be flatten for computational simplicity when updating the parent nodes
        orig_annot_shape = np.shape(Annotation)
        flat_annot = np.ndarray.flatten(Annotation, order = 'C')

        # Create a copy of the annotation structure that reduces areas to their parent nodes
        Annotation_copy = np.zeros(np.shape(flat_annot))

        for kid_id,parent_id in kid_to_father.items():
            tmp  = flat_annot == int(kid_id)
            nzero_ids = np.nonzero(tmp)
            Annotation_copy[nzero_ids] = parent_id 

        return Annotation_copy    

## BrainPlotter class
### fit function
### Author: Nestor Timonidis
### Description:   
In order to make a cortical flatmap visualization for the mouse cortex,    
we need to initially ensure that we have reduced the complexity  
of structural brain areas to a layer inspecific level that corresponds    
to clear anatomically distinct areas like the primary visual or motor cortex.   
Therefore we utilize the MouseConnectivityCache tool to get the ontology    
tree of brain areas, then we reduce it by the isocortex and for each    
isocortical area, we take the identity of its father node, or an area  
belonging up in the tree hierarchy.      
Having achieved that, the next step is to replace the identity of each    
area in the annotation volume structure by the father identity.  
* #### Input:
   * projection = primary array corresponding to an experiment as a manual option. Default = None.
   * projection2 = secondary array corresponding to an experiment as a manual option. Default = None.
   * chosen_indices = In case that the input projection array contains less target brain areas than the complete ~ 1300, a list is required to map the reduced elements to their original space of ~1300 elements. Default = None.
   * layer_profile = the layer profiles of interest that will be used to filter the data, defined by their position in the unionized space, in case they are being provided.  

In [3]:
class BrainPlotter(BrainPlotter):
    
    def fit(self, projection = None, layer_profile = None, projection2 = None, 
            chosen_indices = None):  
        
        self.mcc = MouseConnectivityCache(resolution = self.resolution)
        if projection is not None:
                     
            if len(projection) != 1327 and len(np.shape(projection)) != 3 and chosen_indices == None:
                print('the specified projection experiment has been trimmed\n\
                please provide indices linking the trimmed structures to the original ones (chosen_indices field)')
                return -1
            
        
        all_files = os.listdir('./')
        
        # For reducing time complexity, the brain slice templates provided by the Allen Institute's 
        # MouseConnectivityCache tool have already been uploaded and stored as files:
        # average_template_10.nrrd and average_template_25.nrrd. The code commented bellow
        # displays how the templates can be downloaded in a custom fashion.
        
        template_name = 'average_template_' +  str(self.resolution) + '.nrrd'
        annot_name    = 'annotation_'+  str(self.resolution) + '.nrrd'
        
        if template_name in all_files:
            self.template, template_info = nrrd.read(template_name)
        else:
            self.template, template_info = self.mcc.get_template_volume()
        if annot_name in all_files:
            Annotation, annot_info = nrrd.read(annot_name)
        else:            
            Annotation, annot_info = self.mcc.get_annotation_volume()

        # projection density: number of projecting pixels / voxel volume
        
        if projection is not None and len(np.shape(projection)) == 3:
            self.pd = projection  
            self.pd2 = []
            if projection2 is not None and len(np.shape(projection2)) == 3:
                self.pd2 = projection2  
            self.state = 'raw'  
        elif projection is not None and len(np.shape(projection)) != 3:
            
            with open('structures.csv','rb') as fp:
                structure       = readtable(fp) 
                struct_idx_dict = {val:idx for idx,val in enumerate(structure['id'])}
                str_acronyms    = [val for val in structure['acronym']]
             
            self.pd, self.pd2   = FromUnion2Voxel(Annotation, struct_idx_dict, 
                                                  projection, Pred_ConStr = projection2, 
                                                  rem_ind = chosen_indices)
            
            self.state = 'union'
            
        else:    
            experiment_list = [val for val in all_files if 'experiment_' in val]
            file_to_look  = 'experiment_' + str(self.id)
            if file_to_look in experiment_list:
                pathway  = os.path.join('./',file_to_look)
                pd_files = os.listdir(pathway)
                if 'projection_density_' + str(self.resolution) + '.nrrd' in pd_files:
                    self.pd, pd_info = nrrd.read(pathway +  '/projection_density_' + str(self.resolution) + '.nrrd') 
                else:
                    self.pd, pd_info = self.mcc.get_projection_density(self.id)

            else:
                self.pd, pd_info = self.mcc.get_projection_density(self.id)
            
            
            self.state = 'raw'
            self.pd2 = []
        
        self.whole_pd = None   
        self.whole_pd2 = None   
        
        #**************** Another step here would be to reduce to layers *******#
        if layer_profile is not None:
            laminar_prof   = LaminarRegistration(str_acronyms)
            layer_ids      = [idx for idx, val in enumerate(laminar_prof) 
                                               if val == layer_profile]
            self.whole_pd  = self.pd
            self.whole_pd2 = self.pd2
            self.pd        = filter_cortex(struct_idx_dict, layer_ids, 
                                         Annotation, self.pd)
            self.pd2       = filter_cortex(struct_idx_dict, layer_ids, 
                                         Annotation, self.pd2)
        #***********************************************************************#
        
        self.annotation    = Annotation #self.ReduceToParent(Annotation, self.mcc)

NameError: name 'BrainPlotter' is not defined

## BrainPlotter class   
   ### plot_slice function  
   ### author: Nestor Timonidis
   ### description:   
  provides a visualization in the form of a brain slice, given the parameters defined by the init function and a set of other parameters. This is done by merging the experimental data together with a registration nissl template in an rgb form.
*  #### input:  
   * slice_idx = the point in the first dimension of the data (anterior-posterior range), which will be stable for the brain slice to be visualized. Default = 264.
   * mode = mode with two values. None implies selection of slice_idx and 'max' implies selection of the maximum intensity projection along the anterior-posterior axis of the projection data being selected.  
   * savefile = the save file name. Default = 'slice.png'.
       

In [2]:
class BrainPlotter(BrainPlotter):    
    
    def plot_slice(self, slice_idx = 264, savefile = 'slice.png', mode = None):    
          
        #1:
        mask_lpc = np.zeros((np.shape(self.template)[1],np.shape(self.template)[2]))
        laplace(self.template[slice_idx,:,:], mask_lpc)
        
        norm_mask_lpc = mask_lpc/100
        norm_mask_lpc[norm_mask_lpc > 1] = 1
        
        cmap_mask = plt.get_cmap('gray') 
        rgb_mask  = cmap_mask(norm_mask_lpc, bytes = True)
      
        if mode == 'max':
            pd_mip  = self.pd.max(axis=0)
        else:
            pd_mip  = self.pd[slice_idx,:,:]
          
        if self.whole_pd is not None:
            pd_whole_mip = self.whole_pd[slice_idx,:,:]
            pd_mip  = pd_mip/(1.0*np.max(pd_whole_mip))
        elif len(np.unique(self.pd2)) > 1:  
            max1 = np.max(self.pd)
            max2 = np.max(self.pd2)
            if max1 > 5*np.std(self.pd): max1 = np.max(pd_mip)
            if max2 > 5*np.std(self.pd2): max2 = np.max(self.pd2[slice_idx,:,:])
            pd_mip  = pd_mip/(1.0*max(max1,max2))
        else:    
            pd_mip  = pd_mip/(1.0*np.max(pd_mip))
        #3:
        cmap_pd = plt.get_cmap('hot') 
        rgb_pd  = cmap_pd(pd_mip, bytes = True)
        #4:
        new_mat = np.maximum(rgb_pd, rgb_mask)
         
        # Look at a slice from the average template and annotation volume
        def format_func(value, tick_number):
            return str(value * 0.025)
           
        fig  = plt.figure(figsize=(15, 6))
        ax   = plt.axes()
        ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
        ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
        pos = ax.imshow(new_mat, cmap = 'hot', aspect='equal', vmin = 0, vmax = 1)
        fig.colorbar(pos, ax=ax)
        #plt.title(savefile.split('.')[0], fontsize = 20)
        plt.savefig(savefile)
        plt.show()
        
        if len(np.unique(self.pd2)) > 1:
            pd_mip  = self.pd2[slice_idx,:,:]
            if self.whole_pd2 is not None:
                pd_whole_mip = self.whole_pd2[slice_idx,:,:]
                pd_mip  = pd_mip/(1.0*np.max(pd_whole_mip))
            else: 
                pd_mip  = pd_mip/(1.0*max(max1,max2))

            #3:
            cmap_pd = plt.get_cmap('hot') 
            rgb_pd  = cmap_pd(pd_mip, bytes = True)
            #4:
            new_mat = np.maximum(rgb_pd, rgb_mask)

            # Look at a slice from the average template and annotation volume
            def format_func(value, tick_number):
                return str(value * 0.025)

            fig  = plt.figure(figsize=(15, 6))
            ax   = plt.axes()
            ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax.imshow(new_mat, cmap = 'hot', aspect='equal', vmin = 0, vmax = 1)
            #plt.title(savefile.split('.')[0] + '_2', fontsize = 20)
            plt.savefig(savefile.split('.')[0] + '_pred.jpg')
            plt.show()

NameError: name 'BrainPlotter' is not defined

## BrainPlotter class   
   ### plot_flatmap function  
   ### author: Nestor Timonidis
   ### description:   
 provides a visualization in the form of a cortical flatmap, given the parameters defined by the init function and a set of other parameters. This is done by converting the experimental data in a flatmap form (calling the cortical_map_10 function), and then merging them together with a registration nissl template and an annotation template in an rgb form.
* #### input:  
   * savefile = the save file name. Default = 'slice.png'.
   * annot_mask_pd = the cortical flatmap merged with with a registration nissl template and an annotation template in an rgb form.

In [1]:
class BrainPlotter(BrainPlotter):
    
    def plot_flatmap(self,savefile = None, mode = 'continuous'):  
        
        if self.resolution != 10:
           print 'warning! plot_flatmaps currently accepts only 10 micrometer resolution so please change your resolution settings to 10'
           return 
        
        cm1 = cm.CorticalMap(projection='top_view')
        
        #%% scale the nissl template
        #%% initialize colormaps for rgb conversion
        self.template = self.template/(1.0*np.max(self.template))
        cmap_mask     = plt.get_cmap('gray') 
        cmap_pd       = plt.get_cmap('hot') 
        
        # For reasons of time complexity, the trs_annot_lpc_bordered 
        # data structure is being loaded by a .pkl file. However,
        # commented is the MakeArrayBorders function that creates it
        # by masking the Annotation file based on borders and not
        # borders between structural brain areas.
        if 'Annotation_copy2.pkl' in os.listdir('./'):
            self.transformed_annot = pk.load(open('Annotation_copy2.pkl','rb'))
        else:   
            transformed_annot      = cm1.transform(self.annotation, 
                                                    agg_func = np.mean)
            self.transformed_annot = MakeArrayBorders(transformed_annot)
        
        #%% transform projection density and nissl template
        rgb_annot     = cmap_mask(self.transformed_annot, bytes = True)
        trs_pd        = cm1.transform(self.pd, agg_func = np.max)
        if self.whole_pd is not None:
            trs_original = cm1.transform(self.whole_pd, agg_func = np.mean)
            trs_pd       = trs_pd/(1.0*np.max(trs_original))
        elif len(np.unique(self.pd2)) > 1:  
            trs_pd2      = cm1.transform(self.pd2, agg_func = np.max)
            max1         = np.max(self.pd)
            max2         = np.max(self.pd2)
            if max1 > 5*np.std(self.pd): max1 = np.max(trs_pd)
            if max2 > 5*np.std(self.pd2): max2 = np.max(trs_pd2)
            trs_pd       = trs_pd/(1.0*max(max1,max2))    
        else:    
            max1   = np.max(trs_pd)
            if max1 > 5*np.std(self.pd): max1 = np.max(trs_pd)
            trs_pd = trs_pd/(1.0*max1)
            
        if mode == 'binary':
            trs_pd[trs_pd <= 0.5] = 0
            trs_pd[trs_pd > 0.5]  = 1
            
        trs_mask      = cm1.transform(self.template, agg_func = np.mean)
        
        #%% convert projection density and nissl template to rbg images
        rgb_mask      = cmap_mask(trs_mask, bytes = True)
        rgb_pd        = cmap_pd(trs_pd, bytes = True)
        
        #%% mix rgb images - projection density with nissl template and annotation
        if self.state == 'union':
            # gray background
            pdZero = trs_mask == 0
            mask_and_pd = rgb_pd.copy()
            mask_and_pd[:,:,0][pdZero] = 128
            mask_and_pd[:,:,1][pdZero] = 128
            mask_and_pd[:,:,2][pdZero] = 128
            nonzeroBorders = self.transformed_annot > 0
            
            for i in range(np.shape(mask_and_pd)[2]):
                mask_and_pd[:,:,i][nonzeroBorders] = 190
            self.out_image  = mask_and_pd
            
        elif self.state == 'raw': 
            annot_pd                = np.maximum(rgb_pd,rgb_annot)
            annot_mask_pd           = np.maximum(annot_pd,rgb_mask)
            self.out_image          = np.maximum(rgb_pd,rgb_mask) #annot_mask_pd
         
        #%% Save the files
        def format_func(value, tick_number):
            return str(value * 0.01)
        
        fig = plt.figure(figsize=(15, 6))
        ax   = plt.axes()
        ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
        ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
        N = np.shape(self.out_image)[0]
        pos = ax.imshow(self.out_image[150:N-230,:,:])
        if savefile is not None: plt.savefig(savefile)
        plt.show()
        
        
        if len(np.unique(self.pd2)) > 1:
            # compute the maximum intensity projection (along the anterior-posterior axis) of the projection data
            
            if self.whole_pd2 is not None:
                trs_original = cm1.transform(self.whole_pd2, agg_func = np.mean)
                trs_pd2      = trs_pd2/(1.0*np.max(trs_original))
            else:    
                trs_pd2  = trs_pd2/(1.0*max(max1,max2))  
                
            if mode == 'binary':
                trs_pd2[trs_pd2 <= 0.5] = 0
                trs_pd2[trs_pd2 > 0.5]  = 1   
                
            rgb_pd                  = cmap_pd(trs_pd2, bytes = True)
            #%% mix rgb images - projection density with nissl template and annotation
            if self.state == 'union':
                # blue background
                mask_and_pd = rgb_pd.copy()
                mask_and_pd[:,:,0][pdZero] = 128
                mask_and_pd[:,:,1][pdZero] = 128
                mask_and_pd[:,:,2][pdZero] = 128
                for i in range(np.shape(mask_and_pd)[2]):
                    mask_and_pd[:,:,i][nonzeroBorders] = 190
                 
                out_image = mask_and_pd
            elif self.state == 'raw': 
                annot_pd                = np.maximum(rgb_pd,rgb_annot)
                annot_mask_pd           = np.maximum(annot_pd,rgb_mask)
                out_image  = annot_mask_pd

            #%% Save the files
            def format_func(value, tick_number):
                return str(value * 0.01)

            plt.figure(figsize=(15, 6))
            ax   = plt.axes()
            ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
            N = np.shape(out_image)[0]
            ax.imshow(out_image[150:N-230,:,:])
            #ax.imshow(out_image)
            if savefile is not None: plt.savefig(savefile.split('.')[0] + '_pred.jpg')
            plt.show()
    

NameError: name 'BrainPlotter' is not defined

## BrainPlotter class 
### Call_SBA function
### author: Rembrandt Bakker, Nestor Timonidis
### description: 
Thi function serves as an bridge between the CCP tool and the SBA composer tool. The data of choice are being converted to nifti format and its url is being converted into a command that can be given to the SBA tool for the visualization process to occur.
* #### Input: 
    *  InputData = the input data for visualization.
    *  mode      = two possible modes for the data to be converted are possible: json mode leads to the creation of a json file, usually for analysis statistics, while nifti leads to the creation of a nifti volume. json mode implies that the input data are in proper dictionary form, while nifti implies that the data are in a 3D array form.
* #### Output: 
    *  sbaCommand = the command produced by the function that can be given to SBA as input through the sbaInterface_class interface.

In [22]:
class BrainPlotter(BrainPlotter):  
    
    def Call_SBA(self, InputData, mode):
            
            if isinstance(InputData,dict) == False and mode == 'json':
                print 'Wrong data format for the specified mode'
                return - 1
            
            if isinstance(InputData,np.ndarray) == False and mode == 'nifti': 
                print 'Wrong data format for the specified mode'
                return - 1
            
            if mode == 'nifti':
                nifti_url    = Save2Nifti(InputData, affine_scale = self.resolution*0.001)
                with open(nifti_url, "rb") as fp:
                    sbaCommand = {
                                    "method":"Composer.import",
                                    "params": {
                                      "name": nifti_url.split('.')[0] + '.bas{sba.ABA_v3^corner,PIR,mm}.nii.gz', 
                                      "encoding": 'base64',
                                      "contents": base64.b64encode(fp.read()).decode('utf-8')
                                    }
                                  }
            if mode == 'json':
                if 'bas' in InputData.keys() and 'markers' in InputData.keys() and 'style' in InputData.keys():
                    sbaCommand = {
                        "method":"Composer.scatter3d",
                        'params' : InputData
                    }
                else:
                    print 'Error! the input is not in a proper json form. Please call the Convert2JSON function and repeat.'
                    return -1
                
            return sbaCommand

## FromUnion2Voxel function
### Author: Nestor Timonidis   
### Description:   
This function maps the projection patterns that are defined by a given tract tracing experiment across anatomically distinct brain areas (unionized space), to a volumetric 3D space as specified by an annotation volume provided by the Allen Institute.
*        #### Input:  
         * Annotation = the Annotation identities of the structures in the volumetric space, as provided by the Allen Institute.
         * struct_idx_dict = dictionary that maps identities of anatomically distinct structures, as provided by the Allen Institute, to their position in a unionized space, meaning a 2D space of anatomical structures and brain related experiments.
         * ConStr = the tract tracing experiment defined in the unionized space, measured by normalized projection density.
         * Pred_ConStr = the equivalent tract tracing experiment as ConStr, but with values being predicted by the the gene expression based predictive model (MesoconnectomePredictor object).
         * rem_ind = In case that the input projection array contains less target brain areas than the complete ~ 1300, a list is required to map the reduced elements to their original space of ~1300 elements. Default = None. 
*        #### Output:   
         * UnionToVoxel      =  a 3D array containing the tracing data of the original experiment mapped to the volumetric space.
         * UnionToVoxel_pred =  a 3D array containing the tracing data of the predicted experiment mapped to the volumetric space.

In [23]:
def FromUnion2Voxel(Annotation, struct_idx_dict, ConStr, Pred_ConStr = None, rem_ind = None):
    
    if Pred_ConStr == None:
        Pred_ConStr = np.zeros(np.shape(ConStr))
        
    original_shape       = Annotation.shape
    Annot_flat           = np.ndarray.flatten(Annotation, order = 'C')
    UnionToVoxel         = np.zeros(np.shape(Annot_flat), dtype = np.float32)
    UnionToVoxel_pred    = np.zeros(np.shape(Annot_flat), dtype = np.float32)
  
    if rem_ind != None: 
        rem_ind  = np.asarray(rem_ind) 
        struct_idx_dict_cp = struct_idx_dict.copy()
        for key,value in struct_idx_dict.items():
            if value not in rem_ind:
               del struct_idx_dict_cp[key]
        struct_idx_dict = struct_idx_dict_cp  
    else: 
        rem_ind  = np.asarray([idx for idx in range(len(ConStr))])
     
    for area_id,area_num in struct_idx_dict.items():
        area_mask                    = Annot_flat == int(area_id)
        area_num_reind               = rem_ind    == area_num
        union_values                 = ConStr[area_num_reind]
        union_values2                = Pred_ConStr[area_num_reind]  
        UnionToVoxel[area_mask]      = union_values
        UnionToVoxel_pred[area_mask] = union_values2
    
    UnionToVoxel      = np.reshape(UnionToVoxel, original_shape, order = 'C')
    UnionToVoxel_pred = np.reshape(UnionToVoxel_pred, original_shape, order = 'C')
    
   
    return UnionToVoxel, UnionToVoxel_pred
    

## BinarizeTheVector function
### Author:   Nestor Timonidis    
### Description:   
Given an input vector with
continuous values, the function binarizes
its values based on a threshold value.     
* ### Input:     
     * input_vec = the vector to be binarized,  
     * binThreshold = the desired ratio between the positive and negative class    
* ### Output:   
    * y = the binarized vector.

In [24]:
def BinarizeTheVector(input_vec,binThreshold):
    Q = np.percentile(input_vec, 100-binThreshold, axis = 0)
    y = np.zeros(np.shape(input_vec))
    for idx,val in enumerate(input_vec):
        if input_vec[idx] > Q:
            y[idx] = 1   
    return y 

## CustomCrossval function
### Author:   Nestor Timonidis    
### Description:   
A custom implementation of the cross-validation method, that validates the performance of a supervised learning model by partitioning into disjoint training and testing sets and testing each test set by the model trained by the train set. The difference of this implementation with the custom ones provided by sklearn is that this implementation stores a set of additional parameters at once, as described by the output section. the actual and the predicted values in corresponding lists, stores class , the model parameters and the model coefficients assigned to the trained features based on averaging over the training sets.    
* ### Input:    
     * mdl = the supervised learning model for utility.
     * X = the input dataset or set of independent variables.   
     * y = the dataset labels or dependent variables.
     * cv = the cross validation method as defined by sklearn. 
* ### Output:   
    * probs = assignment probabilities in case of support by the model.
    * preds = list with all predicted data points, assigned to the list based on their corresponding index in the actual data.
    * trues = the list of the actual samples, on one to one correspondance with the preds list.
    * mdl_list = storage of all model parameters.
    * mean_coeffs = the coefficients attributed to the data features, averaged across all training sets.

In [25]:
def CustomCrossval(mdl,X,y,cv):
    
    probs = np.zeros((np.shape(y)[0],2)); preds = np.zeros((np.shape(y)));
    trues = np.zeros((np.shape(y)))
    coeffs = []; mdl_list = []
    for train, test in cv.split(X,y):
        mdl.fit(X[train],y[train])
        mdl_list.append(mdl)
        if hasattr(mdl, 'best_estimator_'):
            if hasattr(mdl.best_estimator_, 'coef_'):
                coeffs.append(mdl.best_estimator_.coef_)
            elif hasattr(mdl.best_estimator_, 'feature_importances_'):
                coeffs.append(mdl.best_estimator_.feature_importances_)
        if hasattr(mdl, 'predict_proba'):        
            probs[test,:] = mdl.predict_proba(X[test])
        preds[test] = mdl.predict(X[test])
        trues[test] = y[test]
    mean_coeffs = np.mean(coeffs,0)        
    return probs, preds, trues, mdl_list, mean_coeffs

## RemoveNanStructs class
### Author:   Nestor Timonidis    
### Description:   
Given a dataset as input, this function removes columns or rows containing NaN (not-a-number), based on whether the percentage of NaN exceeds a given threshold.
*   ### functions:   
       *        #### init:    
                *  #### description:  
                initializes a RemoveNanStructs class object by selecting the default dimension of the dataset for NaN scanning, and the threshold for exclusion.
                *  #### input:  
                      *  dim = dimension for selection.
                      *  nan_thr = the exclusion threshold.
       *        ####   fit:   
                * #### description:   
                Performs the exclusion of rows or columns as described above.
                 *  #### input:  
                       * X = the dataset to be processed.

                 *  #### output:  
                       * X = the dataset after being processed.
                       * nan_list = list containing the rows or columns that were excluded.

In [26]:
class RemoveNanStructs():
    __slots__ = ['dim','nan_thr']
    def __init__(self, dim = 0, nan_thr = 0.1):
        self.nan_thr = nan_thr
        self.dim = dim
    def fit(self,X):
        nan_list = []
        if self.dim == 0:    #row
            for idx,row in enumerate(X):
                nancols = [col for col in row if mt.isnan(col) == True]
                if len(nancols) >= len(row)*self.nan_thr: # row for deletion spotted
                    nan_list.append(idx)
        elif self.dim == 1:   #column
            for idy in range(len(X[0])):
                nanrows = [X[idx,idy] for idx in range(len(X)) if mt.isnan(X[idx,idy]) == True]
                if len(nanrows) >= len(X[0])*self.nan_thr: # column for deletion spotted
                    nan_list.append(idy)
        X = np.delete(X,nan_list,self.dim)
        return X,nan_list

## BimodalImputation class
### Author:   Nestor Timonidis    
### Description:   
Given a vector that is partitioned into two classes, imputes its missing values by sampling from the two classes according to their distribution.   
*   ### functions:   
       *        #### init:    
                *  #### description:  
                initializes a BimodalImputation class object by selecting the default class for selection as defined by mode.
                *  #### input:  
                      *  mode =  the default class for selection. Default = 0.
                     
       *        ####   fit:   
                * #### description:   
                Performs the bimodal imputation as described above.
                 *  #### input:  
                       * y = the vector for imputation.

                 *  #### output:  
                       * y = the imputed vector

In [27]:
class BimodalImputation():
    __slots__ = ['mode']
    def __init__(self, mode = 0):
         self.mode = mode
    def fit(self, y):
        modality = {}
        nan_poses = [idx for idx,val in enumerate(y) if mt.isnan(val) == True]
        modality[0] = [idx for idx,val in enumerate(y) if val == self.mode]
        modality[1] = [idx for idx,val in enumerate(y) if val != self.mode and mt.isnan(val) == False]
        freq    = np.zeros((2,1))
        freq[0] = len(modality[0])/float(len(y))
        freq[1] = len(modality[1])/float(len(y))
        max_pos = [idx for idx in range(2) if freq[idx] == max(freq)][0]
        min_pos = [idx for idx in range(2) if freq[idx] == min(freq)][0]

        for nan in nan_poses:   # rinse and repeat for all nans
            # Draw a dice
            dice = np.random.random_sample()
            if dice <= min(freq):
               rnd_sl = np.random.randint(low = 0, high = len(modality[min_pos]))
               sample = modality[min_pos][rnd_sl]
            else:
               rnd_sl = np.random.randint(low = 0, high = len(modality[max_pos]))
               sample = y[modality[max_pos][rnd_sl]]
             
            y[nan] = sample

        return y

## Save2Nifti function 
### Author: Nestor Timonidis    
### Description:
This function takes as input volumetric data that converts to nifti volume, as well as converts the volume to url in order to be visualized in the Scalable Brain Composer 3D brain visualization tool.   
* ### Input:  
    * brainData = the voxelized gridded matrix - contains predictions of the connection between each grid and a number of target brain areas. 
    * affine_scale = the scale to be used for the affine transformation matrix. Default = 0.2 for 200 micrometer resolution data.
    * outfile = the savefile of the nifti volume. Default = 'volume_density'
* ### Output:    
    * nifti_url = the urls of the produced nifti volume.

In [28]:
def Save2Nifti(brainData, affine_scale = 0.2, outfile = 'volume_density'):
    
    print 'Saving results to nifti form ............... \n'
    Affine = np.eye(4)
    for row in range(3):
        Affine[row][row] = affine_scale
     
    density_img = nib.Nifti1Image(brainData, Affine)
    outfile_revisited = str(outfile) + '.nii.gz'
    nib.save(density_img, outfile_revisited)
    nifti_url = outfile_revisited
       
    print 'Finished converting nifti files to url form ............... \n'    
    return nifti_url

## render_mpl_table function
### author: Nestor Timonidis
### description:   
This function renders for visualizing data in a table form 

In [29]:
def render_mpl_table(data, col_width = 13.0, row_height = 0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None: 
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax

## MesoconnectomePredictor class
### Author: Nestor Timonidis
### Description:   
This class nests a set of function that are involved in the mesoconnectome prediction pipeline.

### init function
### Description:   
   initalizes a MesoconnectomePredictor object by specifying the gene expression (GeneExp) and connectivity strength (ConStr) matrices for analysis.
    A series of data structures that will be used in the analysis are being loaded and update the MesoconnectomePredictor elements (targetprofiles, params, models, ConDict). The data are being loaded for reasons of time efficiency.  The Classes_and_Modules notebook provides comments and descriptions for the functions in which the data were created.     
    The models element stores default or user-given information regarding the regression models that will be used throughout the analysis, their parameters and the evaluation method. The default setup includes Ridge Regression and Random Forest Regressor as regressors, 3-fold cross-validation for generalization performance evaluation and grid-search cv as a method for optimizing the models' hyperparameters.  
 The ConDict is a dictionary that stores connectivity patterns over sets of different tract tracing experiments, as well as additional metadata such as layer profiles and acronyms of the source brain areas.      
 The params dictionary stores information which will be used throughout the analysis.
 It will be passed as a parameter through the functions and updated at various steps.  
 The leaf_keys contain information about target areas that are present on the finest possible level of description. For instance areas called VISp (primary visual area) and VISp l1 (primary visual area, layer 1) could both be found in the dataset, and therefore the leaf_keys are being used to filter out VISp as a non-leaf key in the structure hierarchy since a finer description is present (VISp l1).  
 The targetprofiles dictionary specializes in storage of information regarding the target mouse brain areas with regards to connectivity patterns in the mouse mesoconnectome.
 Its keys are: 
   *           str_acronym = acronyms of target areas/structures
   *           alt laminar profiles = laminar profiles of cortical structures
   *           father               = the brain regions in which each target belongs to   
 
    



In [30]:
class MesoconnectomePredictor:
    
    __slots__ = ['GeneExp','ConStr','params','models', 'targetprofiles', 'ConDict']
    def __init__(self, GeneExp = [], ConStr = [], params = {}, 
                 targetprofiles = OrderedDict(), models = []):
        
        self.GeneExp = GeneExp
        self.ConStr = ConStr
        self.params = params
        self.targetprofiles = targetprofiles
        self.models = models
        
        self.ConDict = pk.load(open('CreLineDict.pkl','rb'))

        if self.models == []:
            
            tol_num = 0.001
            rfc_params = {'max_depth': [10, 60, 100, None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [25,50,100,200],\
                         'oob_score': [True], 'random_state' : [None],\
                         'max_features' : ['auto']}

            ridge_params = {'alpha': [10e-4,10e-3,10e-2,0.5,1,2.5,5,7.5,10,100],
                          'solver' : ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'saga'],
                          'tol': [tol_num], 'max_iter' : [1000],
                           'fit_intercept' : [True, False], 'normalize' : [True,False]}


            self.models = [GridSearchCV(Ridge(), param_grid = ridge_params, 
                                         scoring = 'neg_mean_squared_error', 
                                         cv = 3, n_jobs = -1),
                            RandomForestRegressor(),
                            DummyRegressor()]
      
        if self.targetprofiles == OrderedDict():
            
            with open('structures.csv','rb') as fp:
                structure = readtable(fp)      
            self.targetprofiles['str_acronym'] = [val for val in structure['acronym']]
            self.targetprofiles['alt laminar profiles'] = LaminarRegistration(targetprofiles['str_acronym'])
            self.targetprofiles['thalamic_structures'] = pk.load(open('thalamic_structures.pkl','rb'))
            self.targetprofiles['subcortical_structures'] = pk.load(open('subcortical_structures.pkl','rb'))
            self.targetprofiles = self.LaminarFiltering(self.targetprofiles)
         
             
        if self.params == {}:
            leaves         = pk.load(open('leaf_nodes.pkl','rb'))
            GeneExp        = h5py.File('G_Exp.hdf5', 'r')['dataset1']
            GeneMeta       = pk.load(open('GeneMeta.pkl','r'))
            GeneIds        = np.asarray([val['genes'][0]['entrez_id'] for val in GeneMeta])
            GeneAcros      = np.asarray([val['genes'][0]['acronym'] for val in GeneMeta])

            # storage of leaf-level structures
            leaf_keys = [int(val[0]) for val in leaves]  
            self.params = {'alter' : None, 'fetch' : 'all', 
                           'structure-abbrev' : self.ConDict['wild_type']['structure-abbrev'],\
                          'acronyms': targetprofiles['str_acronym'],\
                          'method': 'regression', 'leaf': True, \
                          'validation' :  KFold(n_splits = 3, shuffle = True),\
                          'prefix': 'Paper4/',\
                          'Gene Acronyms Original': GeneAcros, 
                          'Gene Ids Original': GeneIds,
                          'leaf_keys' : leaf_keys}


## MesoconnectomePredictor class
### GetLayerResolvedArray function
### Author: Nestor Timonidis
### Description:  
This function takes as input a set of anterograde tracer lines with descriptions of their layer and cell-type specific profiles, and constructs a unionized 3D connectivity array, defined by target brain area x source brain area x layer - cell type profile.   
The normalized connection density measure is being used for quantifying connections between areas, that corresponds to the connectivity strength between two areas normalized by their injection and projection volume.
The model used in this analysis is the RegionalizedModel taken from the mouse connectivity models (MCM) tool of the Allen Institute for Brain Science. The link for the documentation of the model can be found here:   
https://mouse-connectivity-models.readthedocs.io/en/latest/modules/generated/mcmodels.models.voxel.RegionalizedModel.html
* ### Input:
    *   mode      = two possible modes for an array mode. voxelized or unionized depending of the interest is on the level of areas or on the level of voxels.
    *   cre_file  = the input file given by the user that contains information about the cre-lines that will be included in the model. Default option is the supplementary table from the Harris et al paper (2018). 
    *   creFilter = a filter for cre lines that will be included in the model. Valid options are:
        * True  = the default case at which the 15 tracer lines used by Harris et al (2018) shall be included.
        * False = all cre-lines of the input file shall be included.
        * list  = any valid python list given by the user with cre-line names as described in the Allen Brain Atlas webpage:   http://connectivity.brain-map.org/transgenic  
    *     InputDict = dictionary containing user-given data for connectivity array construction. Must contain a 2D projection array in one of the InputDict keys, in order to be used as input to the MCM.
    *     rem_ind = In case that the input projection array contains less target brain areas than the complete ~ 1300, a list is required to map the reduced elements to their original space of ~1300 elements. Default = None. 
    *     y_preds = key to look at InputDict in order to obtain the array of interest for analysis.
    *     resolution = the desired resolution for mapping in volumetric space in case that an InputDict has been provided.
* ### Output:
    *        layer_resolved_array = the 3D layer and cell-type resolved array.
    *        model_meta = metadata of the newly created matrix, such as source and target area acronyms and the layer profiles.

In [4]:
class MesoconnectomePredictor(MesoconnectomePredictor):

    def GetLayerResolvedArray(self, mode = 'unionized', 
                              cre_file = 'Supplementary Table 1.csv', 
                              creFilter = True,
                              InputDict = None, 
                              rem_ind = None,
                              data_type = 'y_preds',
                              resolution = 200):
        
        if mode != 'unionized' and mode != 'voxelized':
                print 'Error! wrong mode: please select unionized or voxelized.'
                return -1

        if InputDict is not None and rem_ind is None:
            print 'Error! Please provide an index list for the input dictionary target structures' 
            return -1

        if creFilter == True:
            # Default filtering of cre-lines based on the 15 tracer lines used by Harris et al 2018
            creFilter = ['Syt6-Cre_KI148', 'Ntsr1-Cre_GN220', 'Sim1-Cre_KJ18',
                        'Efr3a-Cre_NO108', 'Chrna2-Cre_OE25', 'A93-Tg1-Cre',  
                        'Tlx3-Cre_PL56', 'Rbp4-Cre_KL100', 'Rorb-IRES2-Cre',
                        'Scnn1a-Tg3-Cre', 'Nr5a1-Cre', 'Sepw1-Cre_NP39',
                        'C57BL/6J', 'Emx1-IRES-Cre', 'Cux2-IRES-Cre'] 


        creDict = {}
        with open(cre_file) as fp:
            buff = csv.reader(fp)
            for idx,row in enumerate(buff):
                 if len(row) > 1: # concatenate the two rows together - error caused by csv transition
                    row[0] =  row[0] + row[1]
                 remains = [x for x in filter(None, row[0].split(';'))]
                 if remains[0].isdigit():
                    if remains[1] in creFilter or creFilter == False:
                        if 'A93' in remains[1]:         # name incoherence between the default creFilter and the cre_file for A93
                            remains[1] = 'A930038C07Rik-Tg1-Cre'
                        if '-' in remains[7]:           #  covers layers 2-6 and therefore is layer inspecific
                            profile = 'layer inspecific'
                        else:    
                            profile = remains[7] + ' ' + remains[8]
                        if profile not in creDict:
                            creDict[profile] = []
                            creDict[profile].append(remains[1])
                        else:                
                            creDict[profile].append(remains[1])
         
        cache = core.VoxelModelCache(manifest_file= 'voxel_model_manifest.json',
                                     ccf_version = 'annotation/ccf_2016/')
        layer_resolved_array = []
        model_meta = {'layer profiles': [], 'source id': [], 'target id': [], 'model' : []}
        model_meta['layer profiles'] = creDict.keys()
        BP_200 = BrainPlotter(resolution = resolution)

        for cur_profile in creDict.keys():
            print 'Current analyzed cre lines are: {} -> {}'.format(cur_profile, creDict[cur_profile])

            # gather injection and projection data from all tracing experiments
            # belonging to the selected cre-line
            
            cre_input = creDict[cur_profile]
            if InputDict is not None:
                struct_ids = None
                
            else:
                struct_ids = [315]

            tracing_data = cache.get_experiment_data(cre = cre_input,
                                                     injection_hemisphere_id  = 3,
                                                     projection_hemisphere_id = 3,
                                                     flip_experiments = True,
                                                     injection_structure_ids  = struct_ids)

            source_voxels  = tracing_data.injection_mask.coordinates
            injections_64  = np.asarray(tracing_data.injections, dtype = np.float64)
            projections_64 = np.asarray(tracing_data.projections, dtype = np.float64)
            source_key = tracing_data.injection_mask.get_key()
            target_key = tracing_data.projection_mask.get_key() 
             
            if InputDict is not None:
                annot_name = 'annotation_{}.nrrd'.format(resolution)
                tracer_size = 0
                concat_mat = []
                for idx in range(len(creDict[cur_profile])):
                    if 'A93' in creDict[cur_profile][idx]: creDict[cur_profile][idx] = 'A93-Tg1-Cre'
                    if 'C57BL/6J' == creDict[cur_profile][idx]: continue  
                    tracer_size += np.shape(InputDict[creDict[cur_profile][idx]][data_type])[1]
                    if concat_mat == []:
                        concat_mat = np.asarray(InputDict[creDict[cur_profile][idx]][data_type])
                    else:    
                        new_mat    = np.asarray(InputDict[creDict[cur_profile][idx]][data_type])
                        concat_mat = np.concatenate((concat_mat,new_mat), axis = 1)
                    
                Annotation, annot_info = nrrd.read(annot_name)
                old_shape  = (np.shape(Annotation)[0],np.shape(Annotation)[1],
                              np.shape(Annotation)[2],
                              tracer_size)
                voxelized_200 = np.zeros((old_shape))
                for j in range(tracer_size):
                    BP_200.fit(concat_mat[:,j],
                               chosen_indices = rem_ind)
                    voxelized_200[:,:,:,j] = BP_200.pd
                new_shape = (old_shape[0]*old_shape[1]*old_shape[2],old_shape[3])
                target_key = Annotation.reshape((new_shape[0],1))
                projections_64 = voxelized_200.reshape(new_shape).transpose()

            # create a voxel-scale model of the mouse connectome given the injection and projection data
            voxelModel    = models.VoxelModel(source_voxels)
            voxelModel.fit((tracing_data.centroids, injections_64), projections_64)

            # generate a voxel-scale connectivity array based on the fit model
            voxel_array = models.voxel.VoxelConnectivityArray.from_fitted_voxel_model(voxelModel)
            
            with open('structures.csv','rb') as fp:
                structure       = readtable(fp) 
                struct_idx_dict = {val:idx for idx,val in enumerate(structure['id'])}
            
            if mode == 'unionized':
                regional_model = models.voxel.RegionalizedModel.from_voxel_array(
                                 voxel_array, source_key, target_key, dataframe = True)
                temporary      = regional_model.normalized_connection_density.T
                model_meta['model'].append(regional_model) 
                model_meta['source id'] = list(temporary.columns)
                model_meta['target id'] = list(temporary.index)
                layer_resolved_array.append(np.asarray(temporary))  
                 
            elif mode == 'voxelized':
                model_meta['source id'] = list(source_key)
                model_meta['target id'] = list(target_key)
                layer_resolved_array.append(voxel_array.T)
                
        # creation of a 3D array serving our laminarly resolved connectivity array    
        layer_resolved_array = np.asarray(layer_resolved_array, dtype = np.float32)
        model_meta['target id'] = [self.targetprofiles['str_acronym'][struct_idx_dict[str(val)]] for val in model_meta['target id']]
        model_meta['source id'] = [self.targetprofiles['str_acronym'][struct_idx_dict[str(val)]] for val in model_meta['source id']]
    
        return layer_resolved_array, model_meta

NameError: name 'MesoconnectomePredictor' is not defined

## MesoconnectomePredictor class
### DictionaryDecomposition function
### Author: Nestor Timonidis
### Description:  
This function takes as input a data structure and performs dictionary decomposition based on the Dictionary Learning and Sparse Coding algorithm proposed by (insert link here).
* ### Input:
    *   X        = the input data structure.
* ### Output:
    *       Atom = the decomposed atoms (features x independent components).
    *       Code = the decomposed code (samples x independent components). 
    *       dlsc = the trained model.

In [1]:
class MesoconnectomePredictor(MesoconnectomePredictor):

    def DictionaryDecomposition(self, X, n_comp = 200, y = None, x = None,
                                alpha = 1.0, transform_alpha = None, state = None):
        
        print 'Commencing dictionary decomposition ...'
        ErrorMat = []
        alpha_zone = [0.01, 0.05, 0.1, 0.3, 0.5, 1.0, 2.0, 10]
        comp_zone = [2, 10, 25, 50, 75, 100, 150, 200]
       
        dlsc_params = {'alpha': alpha_zone,
                       'n_components': comp_zone,
                       'fit_algorithm': ['lars', 'cd'],
                       'transform_algorithm': ['lasso_cd', 
                                               'lars', 'omp']
                      }
        if y == None:
            y = np.zeros((n_comp, len(X[0])))
        x = np.zeros((len(X),n_comp))  
        print np.shape(x), np.shape(y)
        dlsc  = decomp.DictionaryLearning(n_components = n_comp, alpha = alpha, 
                                          positive_code = True, 
                                          positive_dict = True, 
                                          fit_algorithm = 'cd', 
                                          transform_algorithm = 'lasso_cd', 
                                          transform_alpha = transform_alpha,
                                          n_jobs = -1,
                                          dict_init = y,
                                          code_init = x,
                                          random_state = state)
      
          
        start = time.time()
        dlsc.fit(X)
        Atoms = dlsc.components_
        Code  = dlsc.transform(X)
        sparsity = len([row for row in Atoms for col in row if col == 0])/(1.0*len(Atoms)*len(Atoms[0]))
        X_hat = np.dot(Code,Atoms)
        RecErr = 0.5*np.linalg.norm(X - X_hat, 'fro')
        Code_set_len = len(Code[0])
       
        print 'Atoms : '   + str(Atoms.shape)
        print 'Code : '    + str(Code.shape)
        print 'Sparsity: {}'.format(np.median([len(Code[:,idx][Code[:,idx] > 0])/(1.0*len(Code)) for idx in range(Code_set_len)]))
        print 'Participation: {}'.format(np.median([len(Atoms[idx,:][Atoms[idx,:] > 0])/(1.0*len(Atoms[0])) for idx in range(Code_set_len)]))
        print 'Error: '    + str(RecErr)
        end   = time.time()
        print  'Elapsed time: ' + str((end - start)/60)
        
        return Atoms, Code, dlsc

NameError: name 'MesoconnectomePredictor' is not defined

## MesoconnectomePredictor class
### LaminarFiltering function
### Author: Nestor Timonidis
### Description:  
This function takes as input data describing the acronyms of target brain areas and extracts the information relevant to their layer specificity.
* ### Input:
    *   targetprofiles = targetprofiles =  dictionary that specializes in storage of information regarding the target mouse brain areas with regards to connectivity patterns in the mouse mesoconnectome. 
* ### Output:
    *   targetprofiles = the input dictionary updated with layer specific information regarding the target areas.

In [33]:
class MesoconnectomePredictor(MesoconnectomePredictor):
    
    def LaminarFiltering(self,targetprofiles):
        
        other_profiles = ['polymorph layer', 'pyramidal layer', 'zonal layer',\
                              'optic layer', 'superficial gray', 'granule cell',\
                              'molecular layer', 'outer plexiform', 'glomerular layer',
                              'mitral layer', 'granular layer', 'Purkinje layer',\
                              'granule layer', 'inner plexiform', 'layers 1-3', \
                              'layer 13', 'layer 4/5', 'layer 1-4', 'layer 5/6',\
                              'layer 1-2', 'layer 6', 'layer 1-3']

        #** Fixing the messy strings in laminar_profiles ********#
        key =  'alt laminar profiles'
        for idx,prof in enumerate(targetprofiles[key]):
            if prof.isspace(): targetprofiles[key][idx] = 'layer inspecific'
            elif prof[0].isspace():
                tok = targetprofiles[key][idx].split(' ')
                targetprofiles[key][idx] = tok[1] + ' ' + tok[2]
            if targetprofiles[key][idx] == 'layer 3' \
               or targetprofiles[key][idx] == 'layer 2'\
               or targetprofiles[key][idx] == 'layer 2a'\
               or targetprofiles[key][idx] == 'layer 2b'\
               or targetprofiles[key][idx] == 'layers 3'\
               or targetprofiles[key][idx] == 'layers 2':\
                  targetprofiles[key][idx] = 'layer 2/3'
            elif targetprofiles[key][idx]  == 'layers 1':\
                  targetprofiles[key][idx] = 'layer 1'
            elif targetprofiles[key][idx] in other_profiles:
                 targetprofiles[key][idx] = 'other profiles'
                    
        return targetprofiles                 

## MesoconnectomePredictor class
### PredictivePipeline function
### Author: Nestor Timonidis
### Description:  
In our examples I utilize the unionized anatomical tract tracing data taken from the Allen Institute for Brain Science, as shown in the ParserLibrary notebook. The data come in the form of a dictionary with nested matrices, where each dictionary corresponds to a tracer category like wild type or Syt6-Cre_KI148 cre-line experiments for instance.   
In order to demonstrate the MesoPred tool, a number of categories are selected as shown bellow. This analysis can take more than a day if run for all possible tracer categories. Therefore, for reasons of time efficiency one could skip this cell and move to the next one where the results can be loaded by stored pickle files.  
In this pipeline a number of steps is being followed:  
0.  Conversion of the gene expression data to numpy array format for computational simplicity, initialization of various dictionaries to store analysis related data and results per tracer category (key). Selection of the tracer category (driver) of interest, reduction of tracing data and gene expression data to the leaf-level structures and storage of a number of parameters to be used in the analysis, such as the layer profiles and acronyms of the target areas of the dataset.    
1.  Preprocessing of the gene expression and tract tracing data. The function called in that step is PreProcessing
2.  Prediction with a logistic or linear ridge regression model using the cross-validation method for performance evaluation. The function called in that step is CustomCrossval.
3.  Prediction with a random forest classifier or regressor model using the cross-validation method for performance evaluation. The function called in that step is CustomCrossval.
4.  Prediction with a control (baseline) model using the cross-validation method for performance evaluation. The function called in that step is CustomCrossval.
5.  Storage of the predicted connectivity patterns, probability scores and gene importance (coefficient) scores per tracer in dictionaries stratified by the tested driver categories. The function called in that step is UnravelResults.
6.  Based on the stored connectivity patterns and the actual (ground truth) ones, evaluation of the predictive process with series of statistical measures. Examples include r squared and root mean squared error for regression, and f1-score and area under the roc curve for classification. The function called in that step is Evaluation.
8. Estimate the stability of the fittest model based on the optimal hyperparameters selected for each fold of the CustomCrossval function (step 3). In case that the model is considered as stable, then the model fits the whole data with the optimal hyperparameters.     

### Input:
   *   ConDict = dictionary with connectivity strength patterns per structural brain area used throughout the pipeline.
   *   GeneExp = 2D array with gene expression values per structural brain area used throughout the pipeline.   
   
   
### Output:
   *   ClfResults_ridge = dictionary with predictive results of the main model (ridge in that instance).

In [34]:
class MesoconnectomePredictor(MesoconnectomePredictor):   
    
    def PredictivePipeline(self, ConDict, GeneExp):       
        
        GeneExp = np.asarray(GeneExp, dtype = np.float32)
        ConStr_proc = {}; GeneExp_proc = {}; y = {}
        # matrix reduction to leaf-level structures
        GeneExp  = GeneExp[self.params['leaf_keys'],:] 
        for key in ConDict.keys():
                ## Step 0:
                # measure used not to repeat predictions that have already been done
                outfile = key + '_all_results' 
                ConStr  = np.asarray(ConDict[key]['ConMat'], dtype = np.float32)
                ConStr = ConStr[self.params['leaf_keys'],:]
                self.params['injection_number'] = np.shape(ConStr)[1]  
                self.params['cre-line'] = key
                self.params['structure-abbrev'] = ConDict[key]['structure-abbrev']
                self.params['layer'] = ConDict[key]['layer'][0]
                self.params['cell-type'] = ConDict[key]['cell-type'][0]
                self.params['method'] = 'regression'

                print 'Driver line: '+ str(key) + '\nTracer number: ' + str(ConStr.shape[1]) 
                
                ## Step 1:
                GeneExp_proc[key], sc_gene_exp, ConStr_proc[key], sc_sqrt = self.PreProcessing(
                                                                GeneExp, self.params, ConStr)
                str_copy = np.delete(self.params['acronyms'],self.params['nanCons'],0) 
                str_copy = np.delete(str_copy,self.params['nanGenes'],0)
                pk.dump(self.params, open('updated_params.pkl','wb'))
                start = time.time()
                ## Step 2:
                print 'Ridge ' + self.params['method'] +  ' predictive phase ..'    
                ridge_pred  = list(CustomCrossval(self.models[0], GeneExp_proc[key], 
                                                     ConStr_proc[key], self.params['validation']))
                ridge_pred.append(GeneExp_proc[key])
                #pk.dump(ridge_pred, open(self.params['cre-line'] + '_ridge_pred.pkl', 'wb'))
                print 'Ridge ' + self.params['method'] +  ' has been completed' 

                ## Step 3:
                print 'Random forest ' + self.params['method'] + ' predictive phase ..'
                rf_pred  = list(CustomCrossval(self.models[1], GeneExp_proc[key], 
                                                  ConStr_proc[key], self.params['validation']))
                rf_pred.append(GeneExp_proc[key])                                  
                #pk.dump(rf_pred, open(self.params['cre-line'] + '_rf_pred.pkl','wb'))
                print 'Random forest ' + self.params['method'] +  ' has been completed' 

                ## Step 4:
                print 'Control model ' + self.params['method'] +  ' predictive phase ..'
                bl_pred  = list(CustomCrossval(self.models[2], GeneExp_proc[key], 
                                                  ConStr_proc[key], self.params['validation']))
                bl_pred.append(GeneExp_proc[key])                                       

                #pk.dump(bl_pred, open(self.params['cre-line'] + '_bl_pred.pkl','wb'))
                print 'Control ' + self.params['method'] +  ' has been completed'  

                end = time.time() 
                print  'Elapsed time: ' + str((end - start)/60)   

                ## Step 5:
                print 'Unraveling of results phase ..' 
                ClfResults_ridge = self.UnravelResults(ridge_pred, 
                                                       self.params, sc_sqrt, sc_gene_exp)
                ClfResults_rf    = self.UnravelResults(rf_pred, 
                                                       self.params, sc_sqrt, sc_gene_exp)
                #pk.dump(ClfResults_rf, open(outfile + '_rf.pkl','wb'))
                ClfResults_bl    = self.UnravelResults(bl_pred, 
                                                       self.params, sc_sqrt, sc_gene_exp)
                #pk.dump(ClfResults_bl, open(outfile + '_bl.pkl','wb')) 

                ## Step 6:
                print 'Evaluation of results phase ..'
                self.Evaluation(ClfResults_ridge, ClfResults_rf, ClfResults_bl, self.params) 

                # Step 7:
                unanimity_thr = 0.2
                verdict, params_to_fit = self.StabilityInvestigator(ClfResults_ridge['Model Storage'], unanimity_thr)
                if verdict == 'Full Fit': 
                    final_model = Ridge(**params_to_fit).fit(ridge_pred[5], ridge_pred[2])
                elif verdict == 'Partial Fit': 
                    final_model = [Ridge(**model.best_params_).fit(GeneExp_proc[key], ConStr_proc[key])\
                                   for model in ClfResults_ridge['Model Storage']]

                ClfResults_ridge['final model'] = final_model
                final_model = []
                pk.dump(ClfResults_ridge, open(outfile + '_ridge.pkl','wb'))

                #clear_output()
                
        return ClfResults_ridge

In [ ]:
def render_mpl_table(data, col_width = 13.0, row_height = 0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None: 
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
     
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax

## MesoconnectomePredictor class
### GOenrichment function
### author: Paul Tiesinga, Nestor Timonidis
### description:  
Given a set of genes, this function performs gene ontology enrichment analysis by comparing the set with the org.Mm.eg.db mouse gene database provided by https://bioconductor.org/packages/release/data/annotation/html/org.Mm.eg.db.html. The script utilizes a nested script written in the R programming languages, using the rpy2 library
* ### input:
    * GeneList = the list of genes
* ### output:
    * BP = biological processes related to the gene list based on their statistical significance as indicated by the analysis
    * MF = molecular functions related to the gene list based on their statistical significance as indicated by the analysis
    * CC = cellular components related to the gene list based on their statistical significance as indicated by the analysis

In [1]:
class MesoconnectomePredictor(MesoconnectomePredictor):

    def GOenrichment(self, GeneList):
        # Description: given a set of selected genes, performs enrichment
        #              analysis comparing to a bioconductor database
        import rpy2.robjects as ro
        from rpy2.robjects import numpy2ri
        ro.numpy2ri.activate()
        MGP = ro.r('''
                GOenrichment <-function(GeneList){
                    source("http://bioconductor.org/biocLite.R")
                    lib_path = "~/R/x86_64-pc-linux-gnu-library/3.2/"
                    lib_path_2 = "~/anaconda2/lib/R/library/org.Mm.eg.db"
                    #biocLite("ALL")
                    #biocLite("GOstats", lib = lib_path)
                    #biocLite("Category", lib = lib_path)
                    #biocLite("genefilter", lib = lib_path)
                    #biocLite("org.Mm.eg.db", lib = lib_path_2)
                    #biocLite("gage", lib = lib_path)
                    #biocLite(pkgs=c("Biobase", "IRanges", "AnnotationDbi"),
                     #   suppressUpdates=FALSE,
                      #  suppressAutoUpdate=FALSE,
                       # siteRepos=character(),
                        #ask=TRUE, lib= lib_path)
                    #install.packages("RSQLite")
                    #install.packages("devtools")
                    #require(devtools)
                    #library("RSQLite")
                    #library("gage")
                    #library("genefilter")
                    #library("org.Mm.eg.db")
                    library(DBI)
                    library("GO.db")
                    library("GOstats")
                    library("Category")
                    library("annotate")
                    data(ALL, package="ALL")

                    hgCutoff <- 0.01

                    params <- new("GOHyperGParams",
                           geneIds = GeneList,
                           universeGeneIds = NULL,
                           annotation = "org.Mm.eg.db",
                           ontology = "CC",
                           pvalueCutoff = hgCutoff,
                           conditional = FALSE,
                           testDirection = "over")
                    CC <- hyperGTest(params)
                    sumCC <- data.frame(summary(CC))
                    sumCC <- subset(sumCC, select=c("Term"))

                    foo <- vector(mode="list", length = 3)
                    foo[[3]] <- sumCC

                    return(foo)
            }''')
        r_getname = ro.globalenv['GOenrichment']
        verdict = r_getname(GeneList)
        tmp = np.asarray(verdict[2])[0]
        hits = [element for element in tmp if 'neur' in element or 'synap' in element]

        return hits

NameError: name 'MesoconnectomePredictor' is not defined

## MesoconnectomePredictor class
### LaminarEnrichment function
### Author: Nestor Timonidis
### Description: 
Given a list of published gene markers of cortico-laminar specificity,
this function parses the list and compares an input list of given genes
with the marker list, resulting in estimating frequencies of laminar
profiles strongly enriched in the input list.
* ### Input:
    * gene_list = the input list of genes.
    * infile = the gene marker list.
* ### Output:
    * laminar_hits = frequencies of enriched laminar profiles.

In [36]:
class MesoconnectomePredictor(MesoconnectomePredictor):   
    
    def LaminarEnrichment(self, gene_list, infile = 'Bellgard et al - ISH laminar markers.xls'):
        
        LaminarMarkers = {}
        wb = xlrd.open_workbook(filename = infile)
        ws = wb.sheet_by_index(0)
        keywords = ws.row(0)
        for row_id in range(1,ws.nrows):
            gene_marker = ws.row(row_id)[0].value
            LaminarMarkers[gene_marker] = {}
            for key_id in range(1,7):
                if ws.row(row_id)[key_id].value > 0.4:
                    LaminarMarkers[gene_marker][keywords[key_id].value] = ws.row(row_id)[key_id].value
         
        gene_hits = [val for val in gene_list for val2 in LaminarMarkers.keys() if val.lower() == val2.lower()] 
        laminar_hits = OrderedDict([('Layers 2/3',0), ('Layer 4',0), ('Layer 5',0), ('Layer 6',0), ('Layer 6b',0)])
        for key,value in LaminarMarkers.items():
            if key in gene_hits:
                for laminar_prof in value:
                    laminar_hits[laminar_prof] = laminar_hits[laminar_prof] + 1
        
        return laminar_hits

## MesoconnectomePredictor class
### PlotTheResults function
### author: Nestor Timonidis
### description:  
This function plots and saves the results of the predictive analysis.
* ### input:
    * Results  = the analysis results
    * MetaInfo = information related to the plots. Examples include the size of the plot ticks and the filename for save.


In [37]:
class MesoconnectomePredictor(MesoconnectomePredictor):

    def PlotTheResults(self,Results,MetaInfo):

            k = len(MetaInfo['xtick'])
            fsz = 9
            fig = plt.figure(figsize=(8,6))

            #plt.tight_layout()
            plt.rcParams['figure.figsize']
            plt.boxplot(Results, 0, 'gD', widths = 0.6, whis = [5,95])

            ax = plt.gca()
            ax.xaxis.set_tick_params(labelsize = 14)
            ax.yaxis.set_tick_params(labelsize = 14)
            plt.xticks([i+1 for i in range(k)], MetaInfo['xtick'],rotation = 0)
            plt.text(0.51, MetaInfo['max1'][1], MetaInfo['max1'][0], fontsize = fsz)
            plt.text(0.51, MetaInfo['min1'][1], MetaInfo['min1'][0], fontsize = fsz)
            plt.text(0.51, MetaInfo['med1'][1], MetaInfo['med1'][0], fontsize = fsz)
            plt.text(1.45, MetaInfo['max2'][1], MetaInfo['max2'][0], fontsize = fsz)
            plt.text(1.45, MetaInfo['min2'][1], MetaInfo['min2'][0], fontsize = fsz)
            plt.text(1.45, MetaInfo['med2'][1], MetaInfo['med2'][0], fontsize = fsz)
            plt.text(2.45, MetaInfo['max3'][1], MetaInfo['max3'][0], fontsize = fsz)
            plt.text(2.45, MetaInfo['min3'][1], MetaInfo['min3'][0], fontsize = fsz)
            plt.text(2.45, MetaInfo['med3'][1], MetaInfo['med3'][0], fontsize = fsz)
            plt.title(MetaInfo['title'], fontsize = 14)
            # plt.xlabel(MetaInfo['xlabel'])
            plt.ylabel(MetaInfo['ylabel'], fontsize = 14)
            plt.yticks(np.arange(MetaInfo['lb'] - 0.1, 
                                 MetaInfo['ub'], 0.1))
            #plt.yticks(np.arange(0.4,1.0,0.05))

            plt.savefig(MetaInfo['save_file'])
            plt.show(block = False)
            plt.pause(1)
            plt.close()

## MesoconnectomePredictor class
### Evaluation function
### author: Nestor Timonidis
### description:  
This function plots and saves the results of the predictive analysis.
* ### input: 
    * ClfResults_lr = results related to a linear model (classifier or regressor).
    * ClfResults_rf = results related to a random forest model (classifier or regressor).
    * ClfResults_bl = results related to a control (baseline) model (classifier or regressor).
    * params = parameters related to the evaluation of the analysis. Examples include the abbreviations of the brain structures.

In [3]:
class MesoconnectomePredictor(MesoconnectomePredictor):

    def Evaluation(self, ClfResults_lr, ClfResults_rf, ClfResults_bl, params, measure_list = None):          
          
          MetaInfo = {}
          templ = 'values for predicted tracer - projection patterns\nof driver line:'
          MetaInfo['xlabel'] = 'Classifier Models'
          exclusions = ['Model Storage', 'Gene Scoring', 'Mean Gene Scoring',
                       'y_preds','y_actual','Gene Expression', 'projection scaler',
                       'gene scaler','binary_patterns']
          
          if measure_list is None:
            measure_list = ClfResults_lr.keys()   
          
          for measure in measure_list:
             if measure not in exclusions:
               tmp1 = [val[0] for val in ClfResults_lr[measure]] 
               tmp2 = [val[0] for val in ClfResults_rf[measure]] 
               tmp3 = [val[0] for val in ClfResults_bl[measure]] 
               sort1 = np.argsort(tmp1)[::-1]; sort3 = np.sort(tmp1)[::-1]
               sort2 = np.argsort(tmp2)[::-1]; sort4 = np.sort(tmp2)[::-1]
               sort5 = np.argsort(tmp3)[::-1]; sort6 = np.sort(tmp3)[::-1]
               set_length = len(sort1)   
            
               MetaInfo['min1']  = [params['structure-abbrev'][sort1[set_length-1]], sort3[set_length-1]]
               MetaInfo['max1']  = [params['structure-abbrev'][sort1[0]], sort3[0]]
               MetaInfo['med1']  = [params['structure-abbrev'][sort1[set_length/2]], sort3[set_length/2]]
    
               MetaInfo['min2']  = [params['structure-abbrev'][sort2[set_length-1]], sort4[set_length-1]]
               MetaInfo['max2']  = [params['structure-abbrev'][sort2[0]], sort4[0]]
               MetaInfo['med2']  = [params['structure-abbrev'][sort2[set_length/2]], sort4[set_length/2]]
               MetaInfo['min3']  = [params['structure-abbrev'][sort5[set_length-1]], sort6[set_length-1]]
               MetaInfo['max3']  = [params['structure-abbrev'][sort5[0]], sort6[0]]
               MetaInfo['med3']  = [params['structure-abbrev'][sort5[set_length/2]], sort6[set_length/2]] 
               
               MetaInfo['save_file'] = params['cre-line'] + '_' + measure +'.png'
               MetaInfo['title']     = params['cre-line'] #measure + ' '+ templ + ' ' + params['cre-line']
               MetaInfo['ylabel']    = measure
               MetaInfo['xtick']     = ['Ridge Regression','Random Forest', 'Control']
               if measure == 'AURoc': 
                  MetaInfo['lb'] = 0.4
                  MetaInfo['ub'] = 1.0
               elif measure == 'RMSE' or measure == 'r2': 
                  MetaInfo['lb'] = 0
                  MetaInfo['ub'] = 1.0
               else: 
                  MetaInfo['lb'] = 0.1 
                  MetaInfo['ub'] = 1.0
                
               JoinResults = np.asarray([(a[0],b[0],c[0]) for a,b,c in zip(ClfResults_lr[measure],ClfResults_rf[measure],ClfResults_bl[measure])])
               self.PlotTheResults(JoinResults,MetaInfo) 

NameError: name 'MesoconnectomePredictor' is not defined

##  MesoconnectomePredictor class
### StabilityInvestigator function
### Author: Nestor Timonidis
### Description:  
This function investigates the stability of the results provided by the nested cross-validation method that is being implemented at the CustomCrossval function.
The optimal hyperparameters for all trained nested models are being compared for incoherence. In case of a strong agreement, it is being decided that the predictive results are stable. Otherwise, they are being considered as instable.
* Input:
   * model_list    = the list containing the hyperparameter values per data fold.
   * unanimity_thr = stability is defined over common hypeparameter values between data folds. unanimity_thr is the stability threshold which define in which of the three stability states (see verdict) the model is.
* Output:
   * verdict = message regarding the stability of the model. The three verdicts are: Full Fit, Ensemble Fit or No Fit.
   * params_to_fit = the output parameters over which the whole model is suggested to be trained in case of a full fit. Ensemble fit implies ensemble training over the hypeparameter set of each fold and No Fit implies no fitting at all.

In [39]:
class MesoconnectomePredictor(MesoconnectomePredictor): 
    
    def StabilityInvestigator(self, model_list, unanimity_thr):

        frequency = {}; stability_set = []; params_to_fit = []
        param_dict = [model.best_params_ for model in model_list]
        param_list = [fold.values() for fold in param_dict]

        for param in set(chain.from_iterable(param_list)):
            frequency[param] = len([idx for idx,model_params in enumerate(param_list) if param in model_params])
            if frequency[param] == len(param_dict): stability_set.append(param)

        if len(stability_set)/(1.0*len(np.unique(param_list))) > unanimity_thr:
            print 'model is stable, train the whole dataset with the parameters'
            params_to_fit = param_dict[0]
            verdict = 'Full Fit'
        elif len(stability_set)/(1.0*len(np.unique(param_list))) > unanimity_thr - 0.2:
            verdict = 'Emsemble Fit'
            print 'model is slightly unstable, ensemble training over the cross-validation models'
        else:
            verdict = 'No Fit'
            print 'model is highly unstable, possibly by overfitting'

        return verdict, params_to_fit

## Mesoconnectome Predictor class
### Convert2JSON function
### Author: Nestor Timonidis
### Description:  
Given a dictionary composed of results from the predictive pipeline, this function converts them into JSON format for uses such as export to a .json file or call to the Scalable Brain Composer (SBA) API.  
In order for results to be visualized in the SBA tool, a conversion to json format is necessary.  
Therefore the dictionary bellow indicates the format that results must have in order to be visualized.  
*   Specifically:    
    *    provider    = the name of the tool to be used. sba stands for scalable brain atlas.
    *    atlas       = the Common Coordinate Framework (CCF) version that was used by the Allen Institute to process the raw data that the user has used throughout the analysis. In this case ABA_v3 stands for Allen Brain Atlas version 3, since our data were processed according to that framework.  
    *    orientation = the orientation used in 3D space for the input. RAS stands for: right-anterior-superior.
    *    unit        = the scaling unit. mm stands for milimiters.  
    

*     Input:
        * PredDict = the dictionary composed of predictive results.  
        * ConDict  = the original dictionary with the tract tracing experiments plus additional metadata.
        * pred_measure  = the predictive measure that whose distribution we want to store/visualize.
*     Output:
        * JSonDict = the dictionary that has stored the results in JSON format.

In [40]:
class MesoconnectomePredictor(MesoconnectomePredictor): 
    
    def Convert2JSON(self, PredDict, ConDict, pred_measure = 'r2'):
        
        JSonDict = {'bas': {'provider': "sba",
             'atlas': "ABA_v3",
             'orientation': "RAS",
             'unit': "mm"},
             'style': {}, 'markers':[]} 
        
        if pred_measure not in PredDict[PredDict.keys()[0]].keys():
            print 'Error! measure selected does not exist in the dictionary. Please select a new one'
            return -1
        for tracer_name in PredDict.keys():
            if tracer_name == 'wild_type':
               tracer_layer = ''
               tracer_cell  = ''
            else:
               tracer_layer = ConDict[tracer_name]['layer'][0]
               tracer_cell =  ConDict[tracer_name]['cell-type'][0]
            for idx, measure in enumerate(PredDict[tracer_name][pred_measure]):
                JSonDict['markers'].append({})
                JSonDict['markers'][len(JSonDict['markers'])-1]['value']    = str(measure[0])
                JSonDict['markers'][len(JSonDict['markers'])-1]['position'] = [str(coo) for coo in ConDict[tracer_name]['Coordinates'][idx]]
                JSonDict['markers'][len(JSonDict['markers'])-1]['region']   = ConDict[tracer_name]['structure-abbrev'][idx]
                JSonDict['markers'][len(JSonDict['markers'])-1]['relsize']  = "0.5"
                JSonDict['markers'][len(JSonDict['markers'])-1]['cre-line'] = tracer_name
                if 'layer' in ConDict[tracer_name]:
                    JSonDict['markers'][len(JSonDict['markers'])-1]['layer']  = ConDict[tracer_name]['layer'][idx] 
        
        return JSonDict

## MesoconnectomePredictor class
### PreProcessing function
### author: Nestor Timonidis
### description:  
This function applies a set of pre-processing steps to the input gene expression and connectivity strength data, given a set of parameters. The steps are as follows:
*  1: Removal of structures dominated by NaN (not-a-number) values.
*  2: For gene expression data, imputation of the rest of NaN values by the median value of their corresponding column (feature) vector. For the connectivity data, sampling per column (feature) vector based on the frequency of zero elements and non-zero ones. A custom function called for the sampling is the BimodalImputation function that is described in the above cells.
*  3: Scaling both matrices by transforming them based on their cubic square root and then z-scoring them.
*  4: Removing outlier genes for the gene expression data based on their interquartile range. The function being called for that purpose is the RemoveOutliers function.
*  5: Binarizing the connectivity strength matrix based on the 66th percentile per column (source brain area).

* ### input: 
    * GeneExp = gene expression matrix.
    * ConStr = connectivity strength matrix.
    * params = information related to the pre-processing steps. Examples include removed structures (restCons, nanGenes) and gene acronyms. 
* ### output:
    * GeneExp_hat_ctr = Processed Gene Expression matrix, with a cubic square root transformation and z-scoring.
    * ConStr_sqrt_ctr = Processed Connectivity Strength matrix, with a cubic square root transformation and z-scoring.
    * sc_gene_exp     = the scaler used to z-score the Gene Expression array.
    * sc_sqrt         = the scaler used to z-score the Connectivity Strength array.

In [ ]:
class MesoconnectomePredictor(MesoconnectomePredictor):

    def PreProcessing(self, GeneExp = None, params = None, ConStr = None):
        
        if ConStr is None:
            ConStr = np.zeros(np.shape(GeneExp))
            
        if GeneExp is None:
            GeneExp = np.zeros(np.shape(ConStr))
         
        # ************ Step 1: Cleaning the dataset from rows containing complete nans *******************#
        print 'Data cleaning phase ...\n'
        print 'Size before data cleaning {} - {}'.format(np.shape(GeneExp), np.shape(ConStr))
        original_target_size = len(ConStr)
        ConStr,nanCons = RemoveNanStructs(0,0.1).fit(ConStr)
        print 'Size after intermediate cleaning ' + str(np.shape(ConStr))
        self.params['restCons'] = [idx for idx in range(len(GeneExp)) if idx not in nanCons]
        GeneExp = np.delete(GeneExp,nanCons,0)
       
        GeneExp,nanGenes = RemoveNanStructs(0,0.03).fit(GeneExp)
        self.params['nanGenes'] = nanGenes; params['nanCons'] = nanCons
        self.params['restGenes'] = [idx for idx in range(len(ConStr)) if idx not in nanGenes]
        ConStr = np.delete(ConStr,nanGenes,0)
        print 'Size after data cleaning ' + str(np.shape(ConStr))  
        if original_target_size == 1327:
            tmp = params['restCons']
        elif original_target_size <= len(params['leaf_keys']):
            tmp = [params['leaf_keys'][val] for val in params['restCons']]
            
        self.params['remaining_indices']   = [tmp[val] for val in params['restGenes']]
        self.params['str_cpy'] = [val for idx, val in enumerate(params['acronyms']) if idx in params['remaining_indices']]
        # ************** Impute the value of the remaining nans with column averaging **************#
        print 'Data imputation phase ...\n'
        for task in range(len(ConStr[0])):
            nans = [val for val in ConStr[:,task] if mt.isnan(val) == True]
            if len(nans) > 0:
                ConStr[:,task] = BimodalImputation().fit(ConStr[:,task])

        GeneExp = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0, verbose=0, copy=True).fit_transform(GeneExp) 
        
        print 'Data normalization phase ....\n'
        GeneExp_hat = np.power(GeneExp, 1./3)
        sc_gene_exp = StandardScaler()
        sc_gene_exp.fit(GeneExp_hat)
        GeneExp_hat_ctr = sc_gene_exp.transform(GeneExp_hat)
        ConStr_sqrt = np.power(ConStr, 1./3)
        sc_sqrt     = StandardScaler()
        sc_sqrt.fit(ConStr_sqrt)
        ConStr_sqrt_ctr = sc_sqrt.transform(ConStr_sqrt)
         
        self.params['Gene Acronyms']   = params['Gene Acronyms Original']
        self.params['Gene Ids']        = params['Gene Ids Original'] 
        self.params['rem_genes']       = []
        GeneExp_hat                    = np.delete(GeneExp_hat, params['rem_genes'], 
                                              axis = 1)
        sc_gene_exp.fit(GeneExp_hat)
        
        if len(np.unique(ConStr_sqrt_ctr)) == 1:
            return GeneExp_hat_ctr, sc_gene_exp 
        if len(np.unique(GeneExp_hat_ctr)) == 1:
            return ConStr_sqrt_ctr, sc_sqrt
        
        return GeneExp_hat_ctr, sc_gene_exp, ConStr_sqrt_ctr, sc_sqrt

## MesoconnectomePredictor class
### LaminarRegistration function
### author: Nestor Timonidis
### description:  
Given a set of brain structure acronyms, this function scans the keywords for laminar-specific information and registers structures based on it.
* ### input: 
    * acronyms = brain structure acronyms.
* ### output:
    * laminar_profiles = layer specific profiles attributed to specific brain structures

In [42]:
def LaminarRegistration(acronyms):
 
    laminar_profiles = []
    for acro in acronyms:
        numbers = [idx for idx,val in enumerate(acro) if val.isdigit()]
        if numbers != []: 
            split_letter =  acro[numbers[0]-1]
            split_choices =  acro.split(split_letter)
            potential_choice = split_choices[len(split_choices)-1]
            blobs = [val for val in potential_choice if val.isdigit() == False]
            if len(blobs) < 2 and potential_choice != '' and 'r' not in potential_choice: 
                laminar_profiles.append('layer ' + potential_choice)
            elif '6a' in acro:
                laminar_profiles.append('layer 6a')
            else:
                laminar_profiles.append('layer inspecific')
        else:
            laminar_profiles.append('layer inspecific')
          
    return laminar_profiles   

 ### Convert2ROC function
 ### Author: Nestor Timonidis
 ### Description:
This function selects the most optimal binarization threshold for a set of connectivity patterns of a tracing experiment, based on the area under the ROC curve (AURoc).  
For a given tracing experiment, the predicted patterns are converted in connectivity scores with the use of the logistic sigmoid function. Moreover a set thresholds are being used to binarize the actual patterns.   
Afterwards the AURoc is being applied to the binarized pattern and the connectivity scores. For each experiment, the threshold that is associated with the highest AURoc score is being selected as an indicative threshold for future experiments associated with the given tracer and its source brain area.
### Input:  
*   t_actual = the actual connectivity patterns.
*   t_pred   = the predicted connectivity patterns.  

### Output:
*   cutoff_collector = a dictionary that stores for each tracer the most optimal threshold and the AUROc value associated with it.

In [16]:
class MesoconnectomePredictor(MesoconnectomePredictor):
   
    def Convert2ROC(self,t_actual,t_pred):
         
        from sklearn.metrics import roc_curve
         
        def binarize(val,Q):
            if val < Q:
                return 0
            else:
                return 1
            
        def RocFigure(tpr,fpr, auc, ext_thr, int_thr): 
            plt.figure(figsize = (12,8))
            lw = 2
            for idx in range(len(tpr)):
                #color='darkorange',
                #, label='ROC curve (area = {}, external threshold = {}, internal threshold = {})'.format(np.max(auc), np.argmax(auc[0,:]),opt_thr[np.argmax(auc)])
                plt.plot(fpr[idx], tpr[idx], 
                         lw=lw)
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            ax = plt.gca()
            ax.xaxis.set_tick_params(labelsize = 14)
            ax.yaxis.set_tick_params(labelsize = 14)
            plt.xlabel('False Positive Rate', fontsize = 18)
            plt.ylabel('True Positive Rate', fontsize = 18)
            plt.title('Receiver operating characteristic example', fontsize = 18)
            plt.savefig('paper1_figures/mass_roc_curve_fig.jpg')
            plt.show() 
            print 'ROC curve (area = {}, external threshold = {}, internal threshold = {})'.format(np.max(auc), ext_thr, int_thr)
        
        cutoff_collector = {}
        
        cutoffs = np.floor(np.linspace(0,100,100))
        auc_vec = np.zeros((np.shape(t_actual)[1], len(cutoffs)))
        for tracer in range(np.shape(t_actual)[1]):
            fprs = []; tprs = []
            opt_thrs = np.zeros((len(cutoffs),1))
            for cutoff in cutoffs:
                Q = np.percentile(list(t_actual[:,tracer]), 100-cutoff, axis = 0)
                y_actual = np.vectorize(binarize)(t_actual[:,tracer],Q)
                y_pred   = 1/(1 + np.exp(- t_pred[:,tracer]))
                ones = [val for val in y_actual if val == 1]
                if cutoff >= 20 and cutoff <= 80 and len(np.unique(y_actual)) == 2:
                    auc = metrics.roc_auc_score(y_actual, y_pred, average = 'micro')
                    fpr, tpr, thresholds = roc_curve(y_actual, y_pred)
                    opt_thrs[int(cutoff),0] = thresholds[np.argmax(tpr[1:len(tpr)-1]/fpr[1:len(fpr)-1]) + 1] 
                    #RocFigure(tpr,fpr, auc)]
                    fprs.append(fpr); tprs.append(tpr)
                    auc_vec[tracer,int(cutoff)] = auc
                 
            if tracer == 0:  
                ext_thr = 100 - np.argmax(auc_vec[0,:])
                internal_thr = opt_thrs[np.argmax(auc_vec[0,:]),0]
                RocFigure(tprs,fprs, auc_vec, ext_thr, internal_thr)
            cutoff_collector[tracer] = (cutoffs[np.argmax(auc_vec[tracer,:])], np.max(auc_vec[tracer,:]))    

        return cutoff_collector       
        

NameError: name 'MesoconnectomePredictor' is not defined

## MesoconnectomePredictor class
### UnravelResults function
### author: Nestor Timonidis
### description:  
This function receives a ground truth vector and predictive results from a set of analyses 
and makes statistical evaluations by unraveling the results and comparing them with the ground truth.
* ### input:  
    * predictions = predictive results and ground truth from a set of analyses. 
    * params = information related to the predictive evaluation. Examples include the method category (classification or regression) and acronyms of genes used in the analysis.
    * scaler_con = model used to z-score the connectivity data before the predictive process. During the function restores the actual and predicted results into their original scale.
    * scaler_gene = model used to z-score the gene expression data before the predictive process. During the function restores the actual and predicted results into their original scale.
* ### output:
    * ClfResults = evaluations of the predictive results stored in a dictionary.

In [44]:
class MesoconnectomePredictor(MesoconnectomePredictor):
    
    def UnravelResults(self,predictions, params, scaler_con, scaler_gene):
         
        num      = 1   
        y        = predictions[2]
        y_preds  = predictions[1]
        y_scores = predictions[0]
        mdls     = predictions[3]
        coeffs   = predictions[4]
        y_preds  = scaler_con.inverse_transform(y_preds)
        y        = scaler_con.inverse_transform(y)
        GeneExpression = predictions[5]
        GeneExpression = scaler_gene.inverse_transform(GeneExpression)
        
        ClfResults = {}
        ClfResults['Model Storage']     = mdls
        ClfResults['RMSE']              = np.zeros((np.shape(y)[num],1))
        ClfResults['r2']                = np.zeros((np.shape(y)[num],1))
        ClfResults['y_actual']          = y 
        ClfResults['y_preds']           = y_preds
        ClfResults['Gene Expression']   = GeneExpression
        ClfResults['projection scaler'] = scaler_con
        ClfResults['gene scaler']       = scaler_gene
            
        if len(predictions) > 4 and np.shape(coeffs)!= ():
            mean_coeffs = np.mean(coeffs,0)
            tmp_mean    = np.argsort(mean_coeffs)[::-1]
            ClfResults['Mean Gene Scoring'] = [params['Gene Acronyms'][tmp_mean],
                                               mean_coeffs[tmp_mean], tmp_mean]
            ClfResults['Gene Scoring'] = []
            for coeff in coeffs:
                tmp = np.argsort(coeff)[::-1]
                ClfResults['Gene Scoring'].append(
                    [params['Gene Acronyms'][tmp],coeff[tmp], tmp])    
 
        for num in range(len(y[0])):
            ClfResults['RMSE'][num]      = metrics.mean_squared_error(y[:,num], y_preds[:,num])
            ClfResults['r2'][num]        = metrics.r2_score(y[:,num], y_preds[:,num])

        return ClfResults

## MesoconnectomePredictor class
### PredictProjections function
### Author: Nestor Timonidis
### Description:   
this function is based on the product of our predictive pipeline (PredictivePipeline function), which provides predictions of tracing projection patterns as mentioned in the Overview section. Therefore, since the predictive models are being stored
in our tool, users can select their data by giving them as input as well as selecting layer profiles and source brain areas of interest, and the predictive model best fitting the input selections will be used to predict the input data. The end result is a connectivity/projection matrix in a dataframe format, with rows corresponding to target brain areas and columns to source areas, as well as an export of the matrix to a csv file.
* ### Input:   
    * InputArray    =  the input array that will be given as input to the models for the predictive process.
    * layer_profile =  filter regarding layer profiles of interest. For instance, a user might be interested in projections from corticothalamic or layer 5 neuronal populations.
* ### Output:
    * ProjectionMat = the predicted projection matrix in dataframe format.
    * exportFile    = the name of the exported csv file.
    * ProjectionDict = dictionary that stores more detailed information about the predicted projections
    
    
    

In [45]:
class MesoconnectomePredictor(MesoconnectomePredictor):   
    
    def PredictProjections(self, InputArray, layer_profile = None):
         
        ProjectionMat  = []
        source_areas   = []
        ModelToUse     = {} 
        ProjectionDict = {}
        
        if 'remaining_indices' not in self.params.keys():
            temp_params = pk.load(open('updated_params.pkl','rb'))
            self.params['remaining_indices'] = temp_params['remaining_indices']
        target_areas  = np.asarray(self.targetprofiles['str_acronym'])[self.params['remaining_indices']]
        
        
        InputArray_scaled = StandardScaler().fit_transform(InputArray)
        
        print 'Restoring predicted models ....'
        for key in self.ConDict.keys():
            if os.path.isfile('./' + key + '_all_results' + '_ridge.pkl') == True:
                ModelToUse[key] = {}
                infile = key + '_all_results' + '_ridge.pkl'
                tmp    = pk.load(open(infile,'r'))
                ModelToUse[key]['model']   = tmp['final model']
                if 'projection scaler' in tmp.keys():
                    ModelToUse[key]['scaler']  = tmp['projection scaler']
                ModelToUse[key]['profile'] = self.ConDict[key]['layer'][0] + ' ' + self.ConDict[key]['cell-type'][0]
                ModelToUse[key]['areas']   = self.ConDict[key]['structure-abbrev']
                                                                                                    
        acceptable_profiles = [value['profile'] for value in ModelToUse.values()]    
        
        # Restore profiles  and brain areas ....
        if layer_profile == 'all':
            selected_models = ModelToUse.keys() 
        elif layer_profile != None:
            if layer_profile != None and layer_profile not in acceptable_profiles:
                print 'Error! layer profile is not properly specified. Acceptable values are:\n{}, {} and {}'.format(acceptable_profiles, None, 'all')
                return -1
            selected_models = [key for key,val in ModelToUse.iteritems() if layer_profile in val['profile']]
        else:
            selected_models = ['wild_type']
              
        # predict the projection patterns
        for key in selected_models:
            temp_areas = ModelToUse[key]['areas']
            ProjectionDict[key] = {}
            for idx,area_to_inspect in enumerate(temp_areas):
                #frequency = np.nonzero(temp_areas == val)
                if area_to_inspect in source_areas:
                    cnt = 2;  
                    while area_to_inspect + ' ' + str(cnt) in source_areas:
                          cnt = cnt + 1  
                    source_areas = source_areas + [area_to_inspect + ' ' + str(cnt)]
                else:
                    source_areas = source_areas + [area_to_inspect] 
                
            temp_mat      = ModelToUse[key]['model'].predict(InputArray_scaled)
            if 'scaler' in ModelToUse[key].keys():
                temp_mat = ModelToUse[key]['scaler'].inverse_transform(temp_mat)  
            
            ProjectionDict[key]['y_preds'] = temp_mat
            if ProjectionMat == []:
                ProjectionMat = temp_mat
            else:    
                ProjectionMat = np.concatenate((ProjectionMat,temp_mat),axis = 1)
                
        
        # convert to dataframe
        ProjectionMat = pd.DataFrame(data = ProjectionMat,index = target_areas, \
                                            columns = source_areas)
        
        # export to csv
        exportFile = 'predicted projections from {} sources.csv'.format(layer_profile)
        ProjectionMat.to_csv(path_or_buf = exportFile)
        
        clear_output()
        
        return ProjectionMat, exportFile, ProjectionDict

## MesoconnectomePredictor class
### Source_statistics function
### author: Nestor Timonidis
### description:  
Given the predictive pipeline results, this function makes summary statistics regarding projection volumes to specific layer profiles for all given measured and predicted tract tracing data.
* ### input:  
    * ClfResults = dictionary containing the predictive pipeline results.
    * tracer_profile =  dictionary that contains information regarding the category a series of tracing experiments.
    * laminar_profiles = dictionary that contains information regarding the layer profiles present or absent in all target brain areas.
    * subcortical_index = indices for target thalamic areas.
    * thalamic_index    = indices for target subcortical areas.
* ### output:
    * proj_summary = layer specific summary statistics for the measured tracing data.
    * proj_pred_summary = layer specific summary statistics for the predicted tracing data.

In [46]:
class MesoconnectomePredictor(MesoconnectomePredictor):
    
    def Source_statistics(self, ClfResults, tracer_profile, laminar_profiles, thalamic_index, subcortical_index):
        # Description: makes summary statistics of predictions for specific
        #              layer to layer projection - pattern predictions

        # predictive statistics per tracer
        projection_stats_true = OrderedDict()
        projection_stats_pred = OrderedDict()
        for tracer in range(np.shape(ClfResults['y_preds'])[1]):

            if tracer_profile[tracer] in projection_stats_pred.keys(): new_key = tracer_profile[tracer] + '_' + str(tracer)
            else: new_key = tracer_profile[tracer]     

            projection_stats_pred[new_key] = OrderedDict()
            projection_stats_true[new_key] = OrderedDict()
            sub_unique = sorted(set(laminar_profiles)) # get the unique values of laminar profiles
            sub_unique.remove('other profiles'); sub_unique.remove('layer inspecific')
            for prof in sub_unique:
                prof_areas  = [idx for idx,val in enumerate(laminar_profiles) if val == prof]
                projection_stats_pred[new_key][prof] = sum(ClfResults['y_preds'][prof_areas,tracer]) 
                projection_stats_true[new_key][prof] = sum(ClfResults['y_actual'][prof_areas,tracer]) 
            projection_stats_pred[new_key]['thalamic'] = sum(ClfResults['y_preds'][thalamic_index,tracer]) 
            projection_stats_pred[new_key]['subcortical'] = sum(ClfResults['y_preds'][subcortical_index,tracer]) 
            projection_stats_true[new_key]['thalamic'] = sum(ClfResults['y_actual'][thalamic_index,tracer]) 
            projection_stats_true[new_key]['subcortical'] = sum(ClfResults['y_actual'][subcortical_index,tracer])    
         
        proj_summary      = np.asarray(pd.DataFrame.from_dict(projection_stats_true,orient='index')).mean(axis = 0)
        kurtosis          = sci.stats.kurtosis(np.asarray(pd.DataFrame.from_dict(projection_stats_true,orient='index')), axis = 0)
        print kurtosis
        proj_summary      /= proj_summary.sum()
        proj_pred_summary = np.asarray(pd.DataFrame.from_dict(projection_stats_pred,orient='index')).mean(axis = 0) 
        proj_pred_summary /= proj_pred_summary.sum()

        return proj_summary, proj_pred_summary

## MesoconnectomePredictor class
### PlotStatistics function
### author: Nestor Timonidis
### description:  
This function calls the Source_statistics function to make layer specific summary statistics regarding the predictive pipeline results. Furthermore, provides visual bargraph plots of the aforementioned statistics.
* ### input:  
    * PredResults    = dictionary containing the predictive pipeline results.
    * laminar_hits   = statistical results from gene laminar enrichment analysis per tracer category.  
    (LaminarEnrichment function).
    * cellular_hits  = statistical results from gene ontology enrichment analysis per tracer category.

In [47]:
class MesoconnectomePredictor(MesoconnectomePredictor):
    
    def PlotStatistics(self, PredResults, structure_summary = False, laminar_summary = False,
                       tracer_category = None, selected_tracer = None, 
                       laminar_hits = None):
        
        if 'nanCons' not in self.params:
            temp_params = pk.load(open('updated_params.pkl','rb'))
            self.params['nanCons'] = temp_params['nanCons']
        if 'remaining_indices' not in self.params:         
            self.params['remaining_indices'] = temp_params['remaining_indices']
        
        all_files = os.listdir('./')
        self.targetprofiles = self.LaminarFiltering(self.targetprofiles)   
        profiles_of_interest = list(np.unique(self.targetprofiles['alt laminar profiles']))
        profiles_of_interest.remove('layer inspecific'); profiles_of_interest.remove('other profiles')
        profiles_of_interest.append('thalamic');profiles_of_interest.append('subcortical')
        
        matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
        laminar_Accuracy = OrderedDict()
        Laminar_Stats = OrderedDict()
        laminar_copy = [self.targetprofiles['alt laminar profiles'][val] for idx,val in enumerate(self.params['remaining_indices'])]
        thalamic_copy = [idx2 for val in self.targetprofiles['thalamic_structures'] for idx2,val2 in enumerate(self.params['remaining_indices']) if val == val2]
        subcortical_copy = [idx2 for val in self.targetprofiles['subcortical_structures'] for idx2,val2 in enumerate(self.params['remaining_indices']) if val == val2]
        proj_summary = {}; proj_pred_summary = {}
        A = None; B = None
        
        if tracer_category == None:
            tracer_category = PredResults.keys()
        else:
            tracer_category = [tracer_category]
            
        for tracer_name in tracer_category:
            print tracer_name
    
            if laminar_summary is not False:  
                laminar_indices = [idx for idx,val in enumerate(laminar_copy) if val in profiles_of_interest]
                tracer_layer    = self.ConDict[tracer_name]['layer'][0]
                tracer_cell     = self.ConDict[tracer_name]['cell-type'][0]
                tracer_profiles = [self.ConDict[tracer_name]['structure-abbrev'][idx] for idx in range(len(self.ConDict[tracer_name]['structure-abbrev']))] 
                xlabel          = profiles_of_interest 
                title           = tracer_name
                proj_summary[tracer_name], proj_pred_summary[tracer_name] = self.Source_statistics(PredResults[tracer_name], \
                                                                                                   tracer_profiles, \
                                                                                                   laminar_copy,
                                                                                                   thalamic_copy,
                                                                                                   subcortical_copy)
                ind = np.arange(len(xlabel))
                ax = plt.subplot(111)
                plt.figure
                ax.bar(ind, proj_summary[tracer_name], width = 0.2, align = 'center')
                ax.bar(ind + 0.2, proj_pred_summary[tracer_name], width = 0.2, align = 'center')
                ax.set_xticklabels(xlabel)
                ax.set_xticks(ind + 0.1)
                ax.xaxis.set_tick_params(labelsize = 18)
                ax.yaxis.set_tick_params(labelsize = 18)
                plt.yticks(np.arange(0, 0.5, step = 0.05))
                plt.title(title, fontsize = 22)
                plt.legend(['predicted','actual'], prop={'size': 20})
                save_name = 'laminar_stats_for_{}.jpg'.format(title)
                plt.savefig(save_name)
                plt.show()
            
            
            #***********************************************************************#
            if structure_summary == True and selected_tracer is not None:
                
                subcortical_structures = pk.load(open('subcortical_structures.pkl', 'rb'))
                cortical_structures    = pk.load(open('cortical_structures.pkl', 'rb'))
                
                cortical_remains_idx = np.asarray([idx for val2 in cortical_structures for idx,val in enumerate(self.params['remaining_indices']) if val == val2])
                subcortical_remains_idx = np.asarray([idx for val2 in subcortical_structures for idx,val in enumerate(self.params['remaining_indices']) if val == val2])
                cortical_remains_acro = [self.targetprofiles['str_acronym'][val] for val2 in cortical_structures for idx,val in enumerate(self.params['remaining_indices']) if val == val2]
                subcortical_remains_acro = [self.targetprofiles['str_acronym'][val] for val2 in subcortical_structures for idx,val in enumerate(self.params['remaining_indices']) if val == val2]
                
                if isinstance(PredResults, (np.ndarray, np.generic)) == True:
                    A = PredResults[:,selected_tracer]/(1.0*np.max(PredResults[:,selected_tracer]))
                    source_name = selected_tracer
                else:    
                    if 'y_actual' in PredResults[tracer_name].keys():
                        max1 = 1.0*np.max(PredResults[tracer_name]['y_actual'][:,selected_tracer])
                        A = PredResults[tracer_name]['y_actual'][:,selected_tracer]/max1
                    if 'y_preds' in PredResults[tracer_name].keys():    
                        max2 = 1.0*np.max(PredResults[tracer_name]['y_preds'][:,selected_tracer])
                        B = PredResults[tracer_name]['y_preds'][:,selected_tracer]/max2
                    source_name = self.ConDict[tracer_name]['structure-abbrev'][selected_tracer]
                
                
                if A is not None:
                    top_cortex_idx   = np.argsort(A[cortical_remains_idx])[::-1][0:30]
                    top_cortex       = np.sort(A[cortical_remains_idx])[::-1][0:30]
                    top_cortex_acros = [cortical_remains_acro[val] for val in top_cortex_idx]

                    plt.figure(figsize = (15,5))
                    ax = plt.gca()
                    plt.xticks(rotation = 25)
                    plt.bar(top_cortex_acros, top_cortex)
                    title = '{} - {} - {} measured'.format(tracer_name,
                                                            source_name,
                                                            'Cortex')
                    plt.title(title)
                    plt.savefig(title + '.jpg')
                    plt.show()

                    top_subcortex_idx   = np.argsort(A[subcortical_remains_idx])[::-1][0:30]
                    top_subcortex       = np.sort(A[subcortical_remains_idx])[::-1][0:30]
                    top_subcortex_acros = [subcortical_remains_acro[val] for val in top_subcortex_idx]

                    plt.figure(figsize = (15,5))
                    plt.xticks(rotation = 25)
                    plt.bar(top_subcortex_acros, top_subcortex)
                    title = '{} - {} - {} measured'.format(tracer_name, 
                                                    source_name, 'Subcortex')
                    plt.title(title)
                    plt.savefig(title + '.jpg')
                    plt.show()
                
                if isinstance(PredResults, (np.ndarray, np.generic)) == False and B is not None:
                    top_cortex_idx   = np.argsort(B[cortical_remains_idx])[::-1][0:30]
                    top_cortex       = np.sort(B[cortical_remains_idx])[::-1][0:30]
                    top_cortex_acros = [cortical_remains_acro[val] for val in top_cortex_idx]

                    plt.figure(figsize = (15,5))
                    ax = plt.gca()
                    plt.xticks(rotation = 25)
                    plt.bar(top_cortex_acros, top_cortex)
                    title = '{} - {} - {} predicted'.format(tracer_name,
                                                    source_name, 'Cortex')
                    plt.title(title)
                    plt.savefig(title + '.jpg')
                    plt.show()

                    top_subcortex_idx   = np.argsort(B[subcortical_remains_idx])[::-1][0:30]
                    top_subcortex       = np.sort(B[subcortical_remains_idx])[::-1][0:30]
                    top_subcortex_acros = [subcortical_remains_acro[val] for val in top_subcortex_idx]

                    plt.figure(figsize  = (15,5))
                    plt.xticks(rotation = 25)
                    plt.bar(top_subcortex_acros, top_subcortex)
                    title = '{} - {} - {} predicted'.format(tracer_name, 
                                                    source_name, 'Subcortex')
                    plt.title(title)
                    plt.savefig(title + '.jpg')
                    plt.show()
            #***********************************************************************#
            if laminar_hits is not None:
                lam_labels = list(np.unique(self.targetprofiles['alt laminar profiles']))
                lam_labels.remove('layer inspecific'); lam_labels.remove('other profiles'); lam_labels.remove('layer 1') 
                plt.figure
                plt.bar(lam_labels, laminar_hits[tracer_name].values(), width = 0.2, align = 'center')
                ax = plt.gca()
                ax.xaxis.set_tick_params(labelsize = 18)
                ax.yaxis.set_tick_params(labelsize = 18)
                plt.title(tracer_name, fontsize =  20)
                plt.xlabel('Enrichment hits', fontsize = 20)
                plt.ylabel('Frequency', fontsize = 20)
                plt.savefig('blabla.png')
                plt.show()


        return proj_summary, proj_pred_summary

##  MesoconnectomePredictor class
### CTEnrichment function
### Author: Nestor Timonidis
### Description: 
compares an input list of known genes with stored lists of known cell-type markers.
### Input:
* gene_list = input list of known genes.   

### Output:
* CT_hits = scores of cell type marker hits.

In [48]:
class MesoconnectomePredictor(MesoconnectomePredictor):   
    
    def CTEnrichment(self, gene_list):
        
        #** Miller files
        Miller = {}
        with open('Miller et al 2014.csv') as fp:
            A = csv.reader(fp)
            for line in A:
                if line[0] == 'Oligodendrocytes':
                    key = 'Oligodendrocytes'
                    Miller[key] = []
                elif line[0] == 'Microglia':
                    key = 'Microglia'
                    Miller[key] = []
                elif line[0] == 'Astrocytes':
                    key = 'Astrocytes'
                    Miller[key] = []
                elif line[0] == 'Neurons':
                    key = 'Neurons'
                    Miller[key] = []
                else: 
                    Miller[key].append(line[0].split(' ')[0].lower())
         
        #** Lein Files
        CTMarkers['Lein_enrichment'] = {}
        keep_outs = [None, 'Non-Expressors']
        wb = xl.load_workbook('Lein et al 2007.xlsx')
        ws = wb['Cell Type-Enriched Markers']

        for j,col in enumerate(list(ws.rows)[0]):
            if col.value not in keep_outs:
               CTMarkers['Lein_enrichment'][col.value] = []
               for i,row in enumerate(ws.rows):
                   if row[j].value not in keep_outs and i > 0 and row[j].value != None:
                      CTMarkers['Lein_enrichment'][col.value].append(row[j].value.lower())
                        
        wb = xlrd.open_workbook('Lein_target_mRNA.xls')
        ws = wb.sheet_by_index(0)

        cell_type_id = 0
        ontology_id  = 1
        gene_id      = 2
        region_id    = 3

        CTMarkers['Lein_targetting'] = {}
        keywords = ['dendrites', 'astrocytes', 'oligodendrocytes']
        for row_id in range(ws.nrows):
            row =  ws.row(row_id)
            hit = [row[0].value for val in keywords if val.lower() in str(row[0].value.lower())]
            if hit != []: 
                new_ctype = hit[0]
                if new_ctype not in CTMarkers['Lein_targetting'].keys(): CTMarkers['Lein_targetting'] [new_ctype] = {'genes': [], 'ontology': []}

            if CTMarkers['Lein_targetting'] != {}:   
                tmp_str  = ''.join(row[gene_id].value)
                tmp_str2 = ''.join(row[ontology_id].value)
                if tmp_str != '':
                    CTMarkers['Lein_targetting'] [new_ctype]['genes'].append(tmp_str.lower())
                if tmp_str2 != '':   
                    CTMarkers['Lein_targetting'] [new_ctype]['ontology'].append(''.join(row[ontology_id].value))

        #** Winden Files
        CTMarkers['Winden'] = {}
        wb = xl.load_workbook('Winden et al 2009.xlsx')
        ws = wb['Sheet1']
        for i,row in enumerate(ws.rows):
            if i> 0:
                if row[3].value not in CTMarkers['Winden'].keys():
                    CTMarkers['Winden'][row[3].value] = []
                CTMarkers['Winden'][row[3].value].append(row[2].value.lower())

        for key in CTMarkers['Winden'].keys():
            if key == 'purple3':
               CTMarkers['Winden']['Cortical'] = CTMarkers['Winden'].pop(key)
            elif key == 'sienna2':
               CTMarkers['Winden']['Somatostatin & Parvalbumin'] = CTMarkers['Winden'].pop(key)
            else:
               CTMarkers['Winden']['Cholecystokinin'] = CTMarkers['Winden'].pop(key)
 
        #** Galazo Files
        CTMarkers['Galazo'] = OrderedDict()
        wb = xl.load_workbook('Galazo et al 2016.xlsx')
        ws = wb['Sheet1']
        CTMarkers['Galazo']['corticothalamic ontologies'] = []
        CTMarkers['Galazo']['corticothalamic genes'] = []
        for i,row in enumerate(ws.rows):
            if i == 0:
                for idx in range(len(row)):
                    CTMarkers['Galazo']['corticothalamic ontologies'].append(row[idx].value)
            else:
                keys = CTMarkers['Galazo'].keys()
                for idx in range(len(row)):
                    tmp = row[idx].value.split(',')
                    CTMarkers['Galazo']['corticothalamic genes'].extend(tmp)
        CTMarkers['Galazo']['corticothalamic genes'] = list(set(CTMarkers['Galazo']['corticothalamic genes']))

        #** Zeisel Files
        keywords = ['pyramidal', 'interneuron']
        CTMarkers['Zeisel'] = OrderedDict()
        store_pos = []
        wb = xl.load_workbook('Zeisel et al 2015.xlsx')
        ws = wb['Modules']
        for i,row in enumerate(ws.rows):
            if i == 0:
                for idx in range(len(row)):
                    hits = [row[idx].value.lower() for val in keywords if val in row[idx].value.lower()]
                    if len(hits) > 0:
                       CTMarkers['Zeisel'][hits[0]] = []
                       store_pos.append(idx)
            elif i > 1:
                keys = CTMarkers['Zeisel'].keys()
                for idx,key in zip(store_pos, keys):
                    if row[idx].value != None:
                       CTMarkers['Zeisel'][key].append(row[idx].value)
        
        
        Markerlist = ['Winden', 'Lein_enrichment', 'Lein_targetting',
                      'Miller', 'Zeisel', 'Galazo']
         
        CT_hits = {}
        for Marker in Markerlist:
            for celltype in CTMarkers[Marker].keys(): 
                if Marker == 'Lein_targetting':
                    geneset  = CTMarkers[Marker][celltype]['genes']
                    key_name = Marker + '- ' + celltype
                elif Marker == 'Galazo':
                    geneset  = CTMarkers[Marker]['corticothalamic genes'] 
                    key_name = Marker + '- ' + 'corticothalamic genes'
                else:    
                    geneset  = CTMarkers[Marker][celltype]
                    key_name = Marker + '- ' + celltype
                 
                gene_hits = [val for val in gene_list for val2 in geneset if val.lower() == val2.lower()] 
                
                if len(gene_hits) > 0:
                    CT_hits[key_name] = gene_hits
                
        return CT_hits       